In [1]:
import altair as alt
from altair.expr import datum, if_
import datetime
from functools import reduce
import json
import numpy as np
import pandas as pd
import psycopg2
pd.options.display.float_format = "{:,.2f}".format
pd.set_option('display.max_columns', 25)

Connect to the database.

In [2]:
with open("config.json") as f:
    conf = json.load(f)
conn = psycopg2.connect(
    dbname=conf["database"],
    user=conf["user"],
    host=conf["host"],
    password=conf["password"]
)
conn.autocommit = True

## Return the donors who gave at least $50,000 to conservative and GOP-aligned groups between Nov. 2 and Dec. 31, 2017, split by their giving before the tax bill's introduction on Nov. 2 and after its introduction. Do this for every off-year going back to the 2010 cycle.

In [3]:
def run_donors_query(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start, end):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_donors_committees_""" + suffix + """ AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN left(contribid, 11) IN ('d1007764625',
                                              'q0001024006',
                                              'U0000000002',
                                              'U0000000003',
                                              'U0000004292',
                                              'U0000004308',
                                              'U0000004652') THEN 'DEVOS'
                 WHEN left(contribid, 11) IN ('U0000003449',
                                              'U0000003450') THEN 'FERTITTA'
                 WHEN left(contribid, 11) IN ('U0000003477',
                                              'U0000003492') THEN 'KOCH'
                 WHEN left(contribid, 11) IN ('h3001352487',
                                              'm0001331786') THEN 'HILDEBRAND'
                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '""" + reference_start + """'
        AND date <= '""" + reference_end + """'
        AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN left(contribid, 11) IN ('d1007764625',
                                                              'q0001024006',
                                                              'U0000000002',
                                                              'U0000000003',
                                                              'U0000004292',
                                                              'U0000004308',
                                                              'U0000004652') THEN 'DEVOS'
                                 WHEN left(contribid, 11) IN ('U0000003449',
                                                              'U0000003450') THEN 'FERTITTA'
                                 WHEN left(contribid, 11) IN ('U0000003477',
                                                              'U0000003492') THEN 'KOCH'
                                 WHEN left(contribid, 11) IN ('h3001352487',
                                                              'm0001331786') THEN 'HILDEBRAND'
                                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE orgname
                             END = gop_donors.match_id
   WHERE date >= '""" + start + """'
     AND date <= '""" + end + """'
     AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '""" + cycle + """'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
  AND match_id NOT IN ('Air Line Pilots Assn',
                       'America First Policies',
                       'American Action Network',
                       'a0000720833',
                       'q0000962134',
                       'Republican Governors Assn',
                       'Southeast Wood Treating')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;


SELECT *
FROM """ + prefix + """_donors_committees_""" + suffix + """;"""
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [4]:
post_bill_donors_committees_17 = run_donors_query("post_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-11-02", "2017-12-31")
pre_bill_donors_committees_17 = run_donors_query("pre_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-01-01", "2017-11-01")

In [5]:
post_bill_donors_committees_15 = run_donors_query("post_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-11-02", "2015-12-31")
pre_bill_donors_committees_15 = run_donors_query("pre_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-01-01", "2015-11-01")

In [6]:
post_bill_donors_committees_13 = run_donors_query("post_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-11-02", "2013-12-31")
pre_bill_donors_committees_13 = run_donors_query("pre_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-01-01", "2013-11-01")

In [7]:
post_bill_donors_committees_11 = run_donors_query("post_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-11-02", "2011-12-31")
pre_bill_donors_committees_11 = run_donors_query("pre_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-01-01", "2011-11-01")

In [8]:
post_bill_donors_committees_09 = run_donors_query("post_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-11-02", "2009-12-31")
pre_bill_donors_committees_09 = run_donors_query("pre_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-01-01", "2009-11-01")

## How much did each contributor give in each period for each cycle?

### 2017

In [9]:
post_bill_donors_17 = post_bill_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_17 = pre_bill_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [10]:
donors_17 = post_bill_donors_17.merge(pre_bill_donors_17, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_17.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_17.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_17["total_pre_bill"].fillna(0, inplace=True)
donors_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 143
Data columns (total 5 columns):
match_id           144 non-null object
organizations      144 non-null object
contributors       144 non-null object
total_post_bill    144 non-null float64
total_pre_bill     144 non-null float64
dtypes: float64(2), object(3)
memory usage: 6.8+ KB


In [11]:
donors_17["pct_post_bill"] = donors_17["total_post_bill"] / (donors_17["total_pre_bill"] + donors_17["total_post_bill"])
donors_17["pct_pre_bill"] = donors_17["total_pre_bill"] / (donors_17["total_pre_bill"] + donors_17["total_post_bill"])
donors_17["change"] = donors_17["total_post_bill"] - donors_17["total_pre_bill"]
donors_17["pct_change"] = (donors_17["total_post_bill"] - donors_17["total_pre_bill"]) / donors_17["total_pre_bill"].abs()
donors_17.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
46,U0000003690,"[U-Line Corp, Uline Inc]","[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH MRS, U...","4,557,300.00","12,051,400.00",0.27,0.73,"-7,494,100.00",-0.62
54,U0000004054,"[Ghpalmer Assoc, GH Palmer Assoc]","[PALMER, GEOFF, PALMER, GEOFFREY H]","1,955,200.00","546,272.00",0.78,0.22,"1,408,928.00",2.58
10,Hillwood Development,[Hillwood Development],[HILLWOOD DEVELOPMENT COMPANY LLC],"1,500,000.00","500,000.00",0.75,0.25,"1,000,000.00",2.00
59,U0000004552,"[Cinemark Holdings, Cinemark USA]","[MITCHELL, LEE, MITCHELL, LEE ROY, MITCHELL, T...","1,007,400.00","131,299.00",0.88,0.12,"876,101.00",6.67
6,FERTITTA,"[Fertitta Entertainment, Station Casinos]","[FERTITTA, FRANK J MR III, FERTITTA, JILL ANN ...","948,000.00","1,408,100.00",0.40,0.60,"-460,100.00",-0.33


#### How much did they give?

In [12]:
print(str(donors_17[donors_17["total_post_bill"] > 0]["match_id"].count()) + " donors gave a combined $" + str(donors_17["total_post_bill"].sum()) + " after the bill was introduced.")
print(str(donors_17[donors_17["total_pre_bill"] > 0]["match_id"].count()) + " donors gave a combined $" + str(donors_17["total_pre_bill"].sum()) + " before the bill was introduced.")

144 donors gave a combined $31106741.0 after the bill was introduced.
104 donors gave a combined $42892030.0 before the bill was introduced.


### 2015

In [13]:
post_bill_donors_15 = post_bill_donors_committees_15.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_15 = pre_bill_donors_committees_15.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [14]:
donors_15 = post_bill_donors_15.merge(pre_bill_donors_15, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_15.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_15.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_15["total_pre_bill"].fillna(0, inplace=True)
donors_15.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 0 to 102
Data columns (total 5 columns):
match_id           103 non-null object
organizations      73 non-null object
contributors       73 non-null object
total_post_bill    73 non-null float64
total_pre_bill     103 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.8+ KB


In [15]:
donors_15["pct_post_bill"] = donors_15["total_post_bill"] / (donors_15["total_pre_bill"] + donors_15["total_post_bill"])
donors_15["pct_pre_bill"] = donors_15["total_pre_bill"] / (donors_15["total_pre_bill"] + donors_15["total_post_bill"])
donors_15["change"] = donors_15["total_post_bill"] - donors_15["total_pre_bill"]
donors_15["pct_change"] = (donors_15["total_post_bill"] - donors_15["total_pre_bill"]) / donors_15["total_pre_bill"].abs()
donors_15.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
5,KOCH,"[Koch Agronomic Services, Koch Industries]","[KOCH, C CHASE MR, KOCH, CHARLES, KOCH, CHARLE...","3,000,000.00",0.00,1.00,0.00,"3,000,000.00",inf
28,U0000003829,[Point72 Asset Management],"[COHEN, STEVE MR]","2,000,000.00","2,002,699.00",0.50,0.50,"-2,699.00",-0.00
24,U0000003682,[Renaissance Technologies],"[MERCER, DIANA, MERCER, DIANA MRS, MERCER, ROB...","560,500.00","14,517,100.00",0.04,0.96,"-13,956,600.00",-0.96
9,U0000000175,[Stephens Inc],"[STEPHENS, WARREN, STEPHENS, WARREN A MR]","549,400.00","3,485,123.00",0.14,0.86,"-2,935,723.00",-0.84
55,d0000677313,"[California Fruit & Tomato Kitchens, Morning S...","[RUFER, CHRIS, RUFER, CHRIS J MR]","510,700.00","155,800.00",0.77,0.23,"354,900.00",2.28


#### How much did they give?

In [16]:
print(str(donors_15[donors_15["total_post_bill"] > 0]["match_id"].count()) + " donors gave a combined $" + str(donors_15["total_post_bill"].sum()) + " after Nov. 1.")
print(str(donors_15[donors_15["total_pre_bill"] > 0]["match_id"].count()) + " donors gave a combined $" + str(donors_15["total_pre_bill"].sum()) + " before Nov. 2.")

67 donors gave a combined $845235.0 after Nov. 1.
100 donors gave a combined $74437576.0 before Nov. 2.


### 2013

In [17]:
post_bill_donors_13 = post_bill_donors_committees_13.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_13 = pre_bill_donors_committees_13.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [18]:
donors_13 = post_bill_donors_13.merge(pre_bill_donors_13, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_13.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_13.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_13["total_pre_bill"].fillna(0, inplace=True)
donors_13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99 entries, 0 to 98
Data columns (total 5 columns):
match_id           99 non-null object
organizations      57 non-null object
contributors       57 non-null object
total_post_bill    57 non-null float64
total_pre_bill     99 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.6+ KB


In [19]:
donors_13["pct_post_bill"] = donors_13["total_post_bill"] / (donors_13["total_pre_bill"] + donors_13["total_post_bill"])
donors_13["pct_pre_bill"] = donors_13["total_pre_bill"] / (donors_13["total_pre_bill"] + donors_13["total_post_bill"])
donors_13["change"] = donors_13["total_post_bill"] - donors_13["total_pre_bill"]
donors_13["pct_change"] = (donors_13["total_post_bill"] - donors_13["total_pre_bill"]) / donors_13["total_pre_bill"].abs()
donors_13.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
24,U0000003903,[Alliance Coal],"[CRAFT, JOE W III, CRAFT, JOSEPH MR III, CRAFT...","515,600.00","162,900.00",0.76,0.24,"352,700.00",2.17
8,U0000000175,[Stephens Inc],"[STEPHENS, WARREN, STEPHENS, WARREN A MR]","442,400.00","51,300.00",0.90,0.10,"391,100.00",7.62
2,DEVOS,"[Alticor Inc, Amway, Amway/Alticor Inc, Amway ...","[DEVOS, DANIEL, DEVOS, DANIEL G, DEVOS, DANIEL...","300,200.00","106,250.00",0.74,0.26,"193,950.00",1.83
21,U0000003690,"[U-Line Corp, Uline Inc]","[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH MRS, U...","261,400.00","533,150.00",0.33,0.67,"-271,750.00",-0.51
17,U0000003498,[Anheuser-Busch],"[BUSCH, AUGUST A III, BUSCH, AUGUST A MR III, ...","89,800.00","91,600.00",0.50,0.50,"-1,800.00",-0.02


#### How much did they give?

In [20]:
print(str(donors_13[donors_13["total_post_bill"] > 0]["match_id"].count()) + " donors gave a combined $" + str(donors_13["total_post_bill"].sum()) + " after Nov. 1.")
print(str(donors_13[donors_13["total_pre_bill"] > 0]["match_id"].count()) + " donors gave a combined $" + str(donors_13["total_pre_bill"].sum()) + " before Nov. 2.")

57 donors gave a combined $2828550.0 after Nov. 1.
95 donors gave a combined $6635388.0 before Nov. 2.


### 2011

In [21]:
post_bill_donors_11 = post_bill_donors_committees_11.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_11 = pre_bill_donors_committees_11.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [22]:
donors_11 = post_bill_donors_11.merge(pre_bill_donors_11, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_11.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_11.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_11["total_pre_bill"].fillna(0, inplace=True)
donors_11.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 95
Data columns (total 5 columns):
match_id           96 non-null object
organizations      66 non-null object
contributors       66 non-null object
total_post_bill    66 non-null float64
total_pre_bill     96 non-null float64
dtypes: float64(2), object(3)
memory usage: 4.5+ KB


In [23]:
donors_11["pct_post_bill"] = donors_11["total_post_bill"] / (donors_11["total_pre_bill"] + donors_11["total_post_bill"])
donors_11["pct_pre_bill"] = donors_11["total_pre_bill"] / (donors_11["total_pre_bill"] + donors_11["total_post_bill"])
donors_11["change"] = donors_11["total_post_bill"] - donors_11["total_pre_bill"]
donors_11["pct_change"] = (donors_11["total_post_bill"] - donors_11["total_pre_bill"]) / donors_11["total_pre_bill"].abs()
donors_11.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
25,U0000004022,[Thiel Capital],"[THIEL, PETER]","1,035,000.00","32,200.00",0.97,0.03,"1,002,800.00",31.14
11,U0000003260,"[Homemaker, Mt Vernon Investments, Winstar Farm]","[TROUTT, KENNETH A, TROUTT, KENNY A MR, TROUTT...","540,800.00","312,400.00",0.63,0.37,"228,400.00",0.73
20,U0000003690,"[U-Line Corp, Uline Inc]","[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH MRS, U...","385,200.00","188,300.00",0.67,0.33,"196,900.00",1.05
30,U0000004393,"[Nevada Restaurant Services, Pere Antoine]","[ESTEY, ALLYSON, ESTEY, PATRICIA RIVERS MS]","102,400.00",0.00,1.00,0.00,"102,400.00",inf
10,U0000003247,"[Charles Schwab & Co, Chas Schwab Brokerage]","[SCHWAB, CHARLES, SCHWAB, CHARLES R, SCHWAB, C...","71,766.00","139,300.00",0.34,0.66,"-67,534.00",-0.48


#### How much did they give?

In [24]:
print(str(donors_11[donors_11["total_post_bill"] > 0]["match_id"].count()) + " donors gave a combined $" + str(donors_11["total_post_bill"].sum()) + " after Nov. 1.")
print(str(donors_11[donors_11["total_pre_bill"] > 0]["match_id"].count()) + " donors gave a combined $" + str(donors_11["total_pre_bill"].sum()) + " before Nov. 2.")

65 donors gave a combined $3178708.0 after Nov. 1.
93 donors gave a combined $7410473.0 before Nov. 2.


### 2009

In [25]:
post_bill_donors_09 = post_bill_donors_committees_09.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_09 = pre_bill_donors_committees_09.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [26]:
donors_09 = post_bill_donors_09.merge(pre_bill_donors_09, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_09.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_09.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_09["total_pre_bill"].fillna(0, inplace=True)
donors_09.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82 entries, 0 to 81
Data columns (total 5 columns):
match_id           82 non-null object
organizations      48 non-null object
contributors       48 non-null object
total_post_bill    48 non-null float64
total_pre_bill     82 non-null float64
dtypes: float64(2), object(3)
memory usage: 3.8+ KB


In [27]:
donors_09["pct_post_bill"] = donors_09["total_post_bill"] / (donors_09["total_pre_bill"] + donors_09["total_post_bill"])
donors_09["pct_pre_bill"] = donors_09["total_pre_bill"] / (donors_09["total_pre_bill"] + donors_09["total_post_bill"])
donors_09["change"] = donors_09["total_post_bill"] - donors_09["total_pre_bill"]
donors_09["pct_change"] = (donors_09["total_post_bill"] - donors_09["total_pre_bill"]) / donors_09["total_pre_bill"].abs()
donors_09.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
1,DEVOS,"[Alticor Inc, Amway, Amway/Alticor Inc, Amway ...","[DEVOS, DANIEL, DEVOS, DANIEL G, DEVOS, DANIEL...","123,300.00","97,300.00",0.56,0.44,"26,000.00",0.27
0,Chickasaw Nation,[Chickasaw Nation],"[CHICKASAW NATION, CHICKASAW NATION, THE, NATI...","75,600.00","96,700.00",0.44,0.56,"-21,100.00",-0.22
43,i3003697615,"[Mpi Research, Northwood Group]","[PARFET, BARBARA A MRS, PARFET, WILLIAM U, PAR...","51,400.00","1,000.00",0.98,0.02,"50,400.00",50.40
36,h1001217591,[Haworth Inc],"[HAWORTH, ETHELYN, HAWORTH, ETHELYN MRS, HAWOR...","40,000.00","2,400.00",0.94,0.06,"37,600.00",15.67
20,U0000003951,[Intercontinentalexchange Inc],"[LOEFFLER, KELLY L MS, SPRECHER, JEFFREY CRAIG...","37,400.00","19,910.00",0.65,0.35,"17,490.00",0.88


#### How much did they give?

In [28]:
print(str(donors_09[donors_09["total_post_bill"] > 0]["match_id"].count()) + " donors gave a combined $" + str(donors_09["total_post_bill"].sum()) + " after Nov. 1.")
print(str(donors_09[donors_09["total_pre_bill"] > 0]["match_id"].count()) + " donors gave a combined $" + str(donors_09["total_pre_bill"].sum()) + " before Nov. 2.")

47 donors gave a combined $760346.0 after Nov. 1.
76 donors gave a combined $1944751.0 before Nov. 2.


## How many contributors gave more money in 2017 than in any prior cycle?

In [29]:
donors_all_years_list = [donors_09, donors_11, donors_13, donors_15, donors_17]

In [30]:
donors_all_years = reduce(lambda left, right: pd.merge(left, right, on="match_id", how="outer"), donors_all_years_list)

In [31]:
donors_all_years.drop(["organizations_x", "organizations_y", "contributors_x", "contributors_y", "change_x", "change_y", "change", "pct_change_x", "pct_change_y",
                       "pct_change"], axis=1, inplace=True)
donors_all_years.columns = ["match_id", "total_post_bill_09", "total_pre_bill_09", "pct_post_bill_09", "pct_pre_bill_09", "total_post_bill_11",
                            "total_pre_bill_11", "pct_post_bill_11", "pct_pre_bill_11", "total_post_bill_13", "total_pre_bill_13", "pct_post_bill_13",
                            "pct_pre_bill_13", "total_post_bill_15", "total_pre_bill_15", "pct_post_bill_15", "pct_pre_bill_15", "organizations", "contributors",
                            "total_post_bill_17", "total_pre_bill_17", "pct_post_bill_17", "pct_pre_bill_17"]
donors_all_years = donors_all_years[["match_id", "organizations", "contributors", "total_post_bill_09", "total_pre_bill_09", "pct_post_bill_09", "pct_pre_bill_09", "total_post_bill_11",
                            "total_pre_bill_11", "pct_post_bill_11", "pct_pre_bill_11", "total_post_bill_13", "total_pre_bill_13", "pct_post_bill_13",
                            "pct_pre_bill_13", "total_post_bill_15", "total_pre_bill_15", "pct_post_bill_15", "pct_pre_bill_15", "total_post_bill_17",
                                    "total_pre_bill_17", "pct_post_bill_17", "pct_pre_bill_17"]]
donors_all_years.fillna(0, inplace=True)
donors_all_years.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 143
Data columns (total 23 columns):
match_id              144 non-null object
organizations         144 non-null object
contributors          144 non-null object
total_post_bill_09    144 non-null float64
total_pre_bill_09     144 non-null float64
pct_post_bill_09      144 non-null float64
pct_pre_bill_09       144 non-null float64
total_post_bill_11    144 non-null float64
total_pre_bill_11     144 non-null float64
pct_post_bill_11      144 non-null float64
pct_pre_bill_11       144 non-null float64
total_post_bill_13    144 non-null float64
total_pre_bill_13     144 non-null float64
pct_post_bill_13      144 non-null float64
pct_pre_bill_13       144 non-null float64
total_post_bill_15    144 non-null float64
total_pre_bill_15     144 non-null float64
pct_post_bill_15      144 non-null float64
pct_pre_bill_15       144 non-null float64
total_post_bill_17    144 non-null float64
total_pre_bill_17     144 non-null flo

In [32]:
print(str(donors_all_years[(donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_15"]) &
                 (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_13"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_11"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_09"])]["match_id"].count()) + " donors increased the share of their annual giving that fell in the last two months of the year in 2017 as compared with every prior odd-numbered year.")
print("Among those donors, " + (str(donors_all_years[(donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_15"]) &
                 (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_13"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_11"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_09"]) &
                                                     (donors_all_years["pct_post_bill_15"] != 0) &
                                                     (donors_all_years["pct_post_bill_13"] != 0) &
                                                     (donors_all_years["pct_post_bill_11"] != 0) &
                                                     (donors_all_years["pct_post_bill_09"] != 0)]["match_id"].count())) + " gave some amount of money in every prior odd-numbered year")

87 donors increased the share of their annual giving that fell in the last two months of the year in 2017 as compared with every prior odd-numbered year.
Among those donors, 7 gave some amount of money in every prior odd-numbered year


In [33]:
donors_17_increased = donors_all_years[(donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_15"]) &
                 (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_13"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_11"]) &
                (donors_all_years["pct_post_bill_17"] > donors_all_years["pct_post_bill_09"])]

## Which of these donors gave the most money in the last two months of the year?

In [34]:
donors_17_increased.sort_values("total_post_bill_17", ascending=False).head(10)

,match_id,organizations,contributors,total_post_bill_09,total_pre_bill_09,pct_post_bill_09,pct_pre_bill_09,total_post_bill_11,total_pre_bill_11,pct_post_bill_11,pct_pre_bill_11,total_post_bill_13,total_pre_bill_13,pct_post_bill_13,pct_pre_bill_13,total_post_bill_15,total_pre_bill_15,pct_post_bill_15,pct_pre_bill_15,total_post_bill_17,total_pre_bill_17,pct_post_bill_17,pct_pre_bill_17
61,U0000004054,"[Ghpalmer Assoc, GH Palmer Assoc]","[PALMER, GEOFF, PALMER, GEOFFREY H]",0.00,"7,000.00",0.00,0.00,"1,000.00","16,000.00",0.06,0.94,0.00,"10,950.00",0.00,0.00,"10,800.00","69,981.00",0.13,0.87,"1,955,200.00","546,272.00",0.78,0.22
110,Hillwood Development,[Hillwood Development],[HILLWOOD DEVELOPMENT COMPANY LLC],0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"50,000.00","50,000.00",0.50,0.50,"1,500,000.00","500,000.00",0.75,0.25
21,U0000004552,"[Cinemark Holdings, Cinemark USA]","[MITCHELL, LEE, MITCHELL, LEE ROY, MITCHELL, T...","30,400.00","4,800.00",0.86,0.14,0.00,"12,500.00",0.00,0.00,0.00,"20,200.00",0.00,0.00,"19,300.00","290,698.00",0.06,0.94,"1,007,400.00","131,299.00",0.88,0.12
114,Pinnacle West Capital,[Pinnacle West Capital],[PINNACLE WEST CAPITAL CORPORATION],0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,269.00,0.00,0.00,"650,000.00","500,000.00",0.57,0.43
122,Metropolitan Milwaukee Assoc of Commerce,[Metropolitan Milwaukee Assoc of Commerce],[METROPOLITAN MILWAUKEE ASSOCIATION OF COMMERC...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"550,000.00",0.00,1.00,0.00
56,U0000003666,"[[24I Contribution], [24T Contribution], Retired]","[JAMES, VIRGINIA, JAMES, VIRGINIA MS]",0.00,"13,200.00",0.00,0.00,"4,500.00","32,500.00",0.12,0.88,"7,800.00","615,400.00",0.01,0.99,"2,700.00","126,200.00",0.02,0.98,"532,400.00","45,500.00",0.92,0.08
4,U0000000074,[Hendricks Holding Co],"[HENDRICKS, DIANE M, HENDRICKS, DIANE MS]","10,500.00","34,150.00",0.24,0.76,"35,800.00","49,300.00",0.42,0.58,"11,800.00","81,000.00",0.13,0.87,"-1,745,800.00","5,033,400.00",-0.53,1.53,"505,400.00","601,000.00",0.46,0.54
139,q0000828404,[Retired],"[PORTER, MICHAEL]",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"500,000.00",0.00,1.00,0.00
52,U0000000337,[Wynn Resorts],"[WYNN, ANDREA, WYNN, ANDREA MRS, WYNN, STEPHEN...",0.00,"4,200.00",0.00,0.00,"61,600.00","55,071.00",0.53,0.47,"84,400.00","140,000.00",0.38,0.62,"72,000.00","271,600.00",0.21,0.79,"455,123.00","124,173.00",0.79,0.21
3,SCHWARZMAN,"[Blackstone Group, None]","[SCHWARTZMAN, STEPHEN A, SCHWARZMAN, CHRISTINE...","4,800.00","7,200.00",0.40,0.60,"5,000.00","13,634.00",0.27,0.73,"24,300.00","104,300.00",0.19,0.81,"57,100.00","901,420.00",0.06,0.94,"415,500.00","411,100.00",0.50,0.50


## How many contributors gave all of their money in the last two months of 2017 while not giving any money in the last two months of any prior cycle?

In [35]:
print(str(donors_all_years[donors_all_years["pct_post_bill_17"] >= 1]["match_id"].count()) + " donors gave all their money in the last two months of 2017. "
+ str(donors_all_years[(donors_all_years["pct_post_bill_17"] >= 1) &
                 (donors_all_years["pct_post_bill_15"] <= 0) &
                (donors_all_years["pct_post_bill_13"] <= 0) &
                (donors_all_years["pct_post_bill_11"] <= 0) &
                 (donors_all_years["pct_post_bill_09"] <= 0)]["match_id"].count()) + " donors gave all their money in the last two months " +
      "of 2017 while not giving any money in the last two months of any prior odd-numbered year.")

39 donors gave all their money in the last two months of 2017. 25 donors gave all their money in the last two months of 2017 while not giving any money in the last two months of any prior odd-numbered year.


In [36]:
donors_17_gave_all_last_two_months = donors_all_years[(donors_all_years["pct_post_bill_17"] >= 1) &
                 (donors_all_years["pct_post_bill_15"] <= 0) &
                (donors_all_years["pct_post_bill_13"] <= 0) &
                (donors_all_years["pct_post_bill_11"] <= 0) &
                 (donors_all_years["pct_post_bill_09"] <= 0)]

## Who were the biggest beneficiaries of this late-year giving in 2017?

In [37]:
post_bill_donors_committees_beneficiaries = post_bill_donors_committees_17.groupby("committee")["total"].sum().reset_index()
post_bill_donors_committees_beneficiaries[post_bill_donors_committees_beneficiaries["total"] >= 1000000].sort_values("total", ascending=False)

,committee,total
160,Republican National Cmte,"5,516,773.00"
37,Congressional Leadership Fund,"2,713,500.00"
166,Restoration PAC,"2,225,000.00"
5,America First Action,"2,000,000.00"
201,Trump Victory,"1,704,400.00"
8,Americas PAC,"1,400,000.00"
193,Texans Are PAC,"1,400,000.00"
180,Senate Leadership Fund,"1,350,000.00"
190,Team Ryan,"1,349,700.00"
7,American Crossroads,"1,025,000.00"


## How many of these donors gave to Democrats in 2017?

In [38]:
def run_democratic_donors_query(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start, end):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_democratic_donors_committees_""" + suffix + """ AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN left(contribid, 11) IN ('d1007764625',
                                              'q0001024006',
                                              'U0000000002',
                                              'U0000000003',
                                              'U0000004292',
                                              'U0000004308',
                                              'U0000004652') THEN 'DEVOS'
                 WHEN left(contribid, 11) IN ('U0000003449',
                                              'U0000003450') THEN 'FERTITTA'
                 WHEN left(contribid, 11) IN ('U0000003477',
                                              'U0000003492') THEN 'KOCH'
                 WHEN left(contribid, 11) IN ('h3001352487',
                                              'm0001331786') THEN 'HILDEBRAND'
                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '""" + reference_start + """'
        AND date <= '""" + reference_end + """'
        AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN left(contribid, 11) IN ('d1007764625',
                                                              'q0001024006',
                                                              'U0000000002',
                                                              'U0000000003',
                                                              'U0000004292',
                                                              'U0000004308',
                                                              'U0000004652') THEN 'DEVOS'
                                 WHEN left(contribid, 11) IN ('U0000003449',
                                                              'U0000003450') THEN 'FERTITTA'
                                 WHEN left(contribid, 11) IN ('U0000003477',
                                                              'U0000003492') THEN 'KOCH'
                                 WHEN left(contribid, 11) IN ('h3001352487',
                                                              'm0001331786') THEN 'HILDEBRAND'
                                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE orgname
                             END = gop_donors.match_id
   WHERE date >= '""" + start + """'
     AND date <= '""" + end + """'
     AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '""" + cycle + """'
WHERE primcode IN ('J1200',
                   'J2100',
                   'J2300',
                   'Z1200',
                   'Z4200',
                   'Z5200',
                   'Z4400')
  AND match_id NOT IN ('Air Line Pilots Assn',
                       'America First Policies',
                       'American Action Network',
                       'a0000720833',
                       'q0000962134',
                       'Republican Governors Assn',
                       'Southeast Wood Treating')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;


SELECT *
FROM """ + prefix + """_democratic_donors_committees_""" + suffix + """;"""
    return pd.read_sql(query, con=conn)

## Return the data for 2017.

In [39]:
post_bill_democratic_donors_committees_17 = run_democratic_donors_query("post_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-11-02", "2017-12-31")
pre_bill_democratic_donors_committees_17 = run_democratic_donors_query("pre_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-01-01", "2017-11-01")

## How much did each contributor give in each period?

In [40]:
post_bill_democratic_donors_17 = post_bill_democratic_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_democratic_donors_17 = pre_bill_democratic_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [41]:
democratic_donors_17 = post_bill_democratic_donors_17.merge(pre_bill_democratic_donors_17, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
democratic_donors_17["total_pre_bill"].fillna(0, inplace=True)
democratic_donors_17["total_post_bill"].fillna(0, inplace=True)
democratic_donors_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30 entries, 0 to 29
Data columns (total 7 columns):
match_id                   30 non-null object
organizations_post_bill    13 non-null object
contributors_post_bill     13 non-null object
total_post_bill            30 non-null float64
organizations_pre_bill     30 non-null object
contributors_pre_bill      30 non-null object
total_pre_bill             30 non-null float64
dtypes: float64(2), object(5)
memory usage: 1.9+ KB


In [42]:
democratic_donors_17["pct_post_bill"] = democratic_donors_17["total_post_bill"] / (democratic_donors_17["total_pre_bill"] + democratic_donors_17["total_post_bill"])
democratic_donors_17["pct_pre_bill"] = democratic_donors_17["total_pre_bill"] / (democratic_donors_17["total_pre_bill"] + democratic_donors_17["total_post_bill"])
democratic_donors_17["change"] = democratic_donors_17["total_post_bill"] - democratic_donors_17["total_pre_bill"]
democratic_donors_17["pct_change"] = (democratic_donors_17["total_post_bill"] - democratic_donors_17["total_pre_bill"]) / democratic_donors_17["total_pre_bill"].abs()
democratic_donors_17.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations_post_bill,contributors_post_bill,total_post_bill,organizations_pre_bill,contributors_pre_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
1,Entertainment Software Assn,[Entertainment Software Assn],[ENTERTAINMENT SOFTWARE ASSOCIATION],"100,000.00",[Entertainment Software Assn],[ENTERTAINMENT SOFTWARE ASSOCIATION],"50,000.00",0.67,0.33,"50,000.00",1.00
2,Shakopee Mdewakanton Sioux Community,[Shakopee Mdewakanton Sioux Community],"[MDEWAKANTON SIOUX COMMUNITY, SHAKOPEE,,,, SHA...","61,700.00",[Shakopee Mdewakanton Sioux Community],"[MDEWAKANTON SIOUX COMMUNITY, SHAKOPEE,,,, SHA...","93,700.00",0.40,0.60,"-32,000.00",-0.34
4,U0000000386,[Avenue Ventures],"[ZUBERI, IMAAD, ZUBERI, IMAAD MR]","47,100.00",[Avenue Ventures],"[ZUBERI, IMAAD, ZUBERI, IMAAD MR]","65,456.00",0.42,0.58,"-18,356.00",-0.28
7,a0000027527,"[Alcalde & Fay, Camival Corp, Carnival Corp, C...","[ARISON, MADELEINE, ARISON, MADELEINE MRS, ARI...","25,000.00","[Alcalde & Fay, Camival Corp, Carnival Corp, C...","[ARISON, MADELEINE, ARISON, MADELEINE MRS, ARI...","40,600.00",0.38,0.62,"-15,600.00",-0.38
9,e1110767126,[Holland Partners],"[HOLLAND, CLYDE, HOLLAND, CLYDE P, HOLLAND, CL...","25,000.00",[Holland Partners],"[HOLLAND, CLYDE, HOLLAND, CLYDE P, HOLLAND, CL...","5,400.00",0.82,0.18,"19,600.00",3.63


#### How much did they give?

In [43]:
print(str(democratic_donors_17["match_id"].count()) + " donors gave a combined $" + str(democratic_donors_17["total_post_bill"].sum() +
                                                                                        democratic_donors_17["total_pre_bill"].sum()) +
      " to liberal-leaning and Democratic groups over the course of 2017. Of that amount, $" +
      str(democratic_donors_17["total_pre_bill"].sum()) + " came before the bill was introduced and $" +
      str(democratic_donors_17["total_post_bill"].sum()) + " came after the bill was introduced.")

30 donors gave a combined $1176191.0 to liberal-leaning and Democratic groups over the course of 2017. Of that amount, $871691.0 came before the bill was introduced and $304500.0 came after the bill was introduced.


## How did each contributor's giving pattern change in each cycle compared with 2017?

### 2015

In [44]:
donors_15_17 = donors_17.merge(donors_15, how="outer", on="match_id", suffixes=["_17", "_15"])
donors_15_17.drop(["change_17", "pct_change_17", "change_15", "pct_change_15"],
            axis=1, inplace=True)
donors_15_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_15_17 = donors_15_17[["match_id", "contributors", "organizations", "total_pre_bill_15", "total_post_bill_15",
                 "pct_pre_bill_15", "pct_post_bill_15", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_15_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 143
Data columns (total 11 columns):
match_id              144 non-null object
contributors          144 non-null object
organizations         144 non-null object
total_pre_bill_15     103 non-null float64
total_post_bill_15    73 non-null float64
pct_pre_bill_15       73 non-null float64
pct_post_bill_15      73 non-null float64
total_pre_bill_17     144 non-null float64
total_post_bill_17    144 non-null float64
pct_pre_bill_17       144 non-null float64
pct_post_bill_17      144 non-null float64
dtypes: float64(8), object(3)
memory usage: 13.5+ KB


In [45]:
donors_15_17["giving_change"] = np.where((donors_15_17["pct_post_bill_17"] > donors_15_17["pct_post_bill_15"]) | (donors_15_17["pct_pre_bill_15"].isnull()), "Increased",
                                  np.where(donors_15_17["pct_post_bill_17"] < donors_15_17["pct_post_bill_15"], "Decreased",
                                           np.where(donors_15_17["pct_post_bill_17"] == donors_15_17["pct_post_bill_15"], "Stayed the same",
                                                   "Other")))
donors_15_17.head(1)

,match_id,contributors,organizations,total_pre_bill_15,total_post_bill_15,pct_pre_bill_15,pct_post_bill_15,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Anschutz Corp,[THE ANSCHUTZ CORPORATION],[Anschutz Corp],nan,nan,nan,nan,0.00,"500,000.00",0.00,1.00,Increased


### 2013

In [46]:
donors_13_17 = donors_17.merge(donors_13, how="outer", on="match_id", suffixes=["_17", "_13"])
donors_13_17.drop(["change_17", "pct_change_17", "change_13", "pct_change_13"],
            axis=1, inplace=True)
donors_13_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_13_17 = donors_13_17[["match_id", "contributors", "organizations", "total_pre_bill_13", "total_post_bill_13",
                 "pct_pre_bill_13", "pct_post_bill_13", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_13_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 143
Data columns (total 11 columns):
match_id              144 non-null object
contributors          144 non-null object
organizations         144 non-null object
total_pre_bill_13     99 non-null float64
total_post_bill_13    57 non-null float64
pct_pre_bill_13       57 non-null float64
pct_post_bill_13      57 non-null float64
total_pre_bill_17     144 non-null float64
total_post_bill_17    144 non-null float64
pct_pre_bill_17       144 non-null float64
pct_post_bill_17      144 non-null float64
dtypes: float64(8), object(3)
memory usage: 13.5+ KB


In [47]:
donors_13_17["giving_change"] = np.where((donors_13_17["pct_post_bill_17"] > donors_13_17["pct_post_bill_13"]) | (donors_13_17["pct_pre_bill_13"].isnull()), "Increased",
                                  np.where(donors_13_17["pct_post_bill_17"] < donors_13_17["pct_post_bill_13"], "Decreased",
                                           np.where(donors_13_17["pct_post_bill_17"] == donors_13_17["pct_post_bill_13"], "Stayed the same",
                                                   "Other")))
donors_13_17.head(1)

,match_id,contributors,organizations,total_pre_bill_13,total_post_bill_13,pct_pre_bill_13,pct_post_bill_13,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Anschutz Corp,[THE ANSCHUTZ CORPORATION],[Anschutz Corp],0.00,"5,000.00",0.00,1.00,0.00,"500,000.00",0.00,1.00,Stayed the same


### 2011

In [48]:
donors_11_17 = donors_17.merge(donors_11, how="outer", on="match_id", suffixes=["_17", "_11"])
donors_11_17.drop(["change_17", "pct_change_17", "change_11", "pct_change_11"],
            axis=1, inplace=True)
donors_11_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_11_17 = donors_11_17[["match_id", "contributors", "organizations", "total_pre_bill_11", "total_post_bill_11",
                 "pct_pre_bill_11", "pct_post_bill_11", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_11_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 143
Data columns (total 11 columns):
match_id              144 non-null object
contributors          144 non-null object
organizations         144 non-null object
total_pre_bill_11     96 non-null float64
total_post_bill_11    66 non-null float64
pct_pre_bill_11       66 non-null float64
pct_post_bill_11      66 non-null float64
total_pre_bill_17     144 non-null float64
total_post_bill_17    144 non-null float64
pct_pre_bill_17       144 non-null float64
pct_post_bill_17      144 non-null float64
dtypes: float64(8), object(3)
memory usage: 13.5+ KB


In [49]:
donors_11_17["giving_change"] = np.where((donors_11_17["pct_post_bill_17"] > donors_11_17["pct_post_bill_11"]) | (donors_11_17["pct_pre_bill_11"].isnull()), "Increased",
                                  np.where(donors_11_17["pct_post_bill_17"] < donors_11_17["pct_post_bill_11"], "Decreased",
                                           np.where(donors_11_17["pct_post_bill_17"] == donors_11_17["pct_post_bill_11"], "Stayed the same",
                                                   "Other")))
donors_11_17.head(1)

,match_id,contributors,organizations,total_pre_bill_11,total_post_bill_11,pct_pre_bill_11,pct_post_bill_11,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Anschutz Corp,[THE ANSCHUTZ CORPORATION],[Anschutz Corp],nan,nan,nan,nan,0.00,"500,000.00",0.00,1.00,Increased


### 2009

In [50]:
donors_09_17 = donors_17.merge(donors_09, how="outer", on="match_id", suffixes=["_17", "_09"])
donors_09_17.drop(["change_17", "pct_change_17", "change_09", "pct_change_09"],
            axis=1, inplace=True)
donors_09_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_09_17 = donors_09_17[["match_id", "contributors", "organizations", "total_pre_bill_09", "total_post_bill_09",
                 "pct_pre_bill_09", "pct_post_bill_09", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_09_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 143
Data columns (total 11 columns):
match_id              144 non-null object
contributors          144 non-null object
organizations         144 non-null object
total_pre_bill_09     82 non-null float64
total_post_bill_09    48 non-null float64
pct_pre_bill_09       48 non-null float64
pct_post_bill_09      48 non-null float64
total_pre_bill_17     144 non-null float64
total_post_bill_17    144 non-null float64
pct_pre_bill_17       144 non-null float64
pct_post_bill_17      144 non-null float64
dtypes: float64(8), object(3)
memory usage: 13.5+ KB


In [51]:
donors_09_17["giving_change"] = np.where((donors_09_17["pct_post_bill_17"] > donors_09_17["pct_post_bill_09"]) | (donors_09_17["pct_pre_bill_09"].isnull()), "Increased",
                                  np.where(donors_09_17["pct_post_bill_17"] < donors_09_17["pct_post_bill_09"], "Decreased",
                                           np.where(donors_09_17["pct_post_bill_17"] == donors_09_17["pct_post_bill_09"], "Stayed the same",
                                                   "Other")))
donors_09_17.head(1)

,match_id,contributors,organizations,total_pre_bill_09,total_post_bill_09,pct_pre_bill_09,pct_post_bill_09,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Anschutz Corp,[THE ANSCHUTZ CORPORATION],[Anschutz Corp],nan,nan,nan,nan,0.00,"500,000.00",0.00,1.00,Increased


## What proportion of donors increased the share of their annual giving that fell in the last two months of 2017 as compared with the same period in each of the previous years?

In [52]:
print(str(donors_15_17[donors_15_17["giving_change"] == "Increased"]["match_id"].count() / donors_15_17["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell in the last two months of the year between 2015 and 2017.")
print(str(donors_13_17[donors_13_17["giving_change"] == "Increased"]["match_id"].count() / donors_13_17["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell in the last two months of the year between 2013 and 2017.")
print(str(donors_11_17[donors_11_17["giving_change"] == "Increased"]["match_id"].count() / donors_11_17["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell in the last two months of the year between 2011 and 2017.")
print(str(donors_09_17[donors_09_17["giving_change"] == "Increased"]["match_id"].count() / donors_09_17["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell in the last two months of the year between 2009 and 2017.")

90.27777777777779% of donors increased the share of their annual giving that fell in the last two months of the year between 2015 and 2017.
84.72222222222221% of donors increased the share of their annual giving that fell in the last two months of the year between 2013 and 2017.
85.41666666666666% of donors increased the share of their annual giving that fell in the last two months of the year between 2011 and 2017.
84.72222222222221% of donors increased the share of their annual giving that fell in the last two months of the year between 2009 and 2017.


## And by how much did these individuals' giving increase?

In [53]:
print("These donors' giving in the last two months of the year increased by $" + str(donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_15"].sum()) + ", from $" + str(donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_15"].sum()) + " to $" + str(donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()) + " between 2015 and 2017.")
print("These donors' giving in the last two months of the year increased by $" + str(donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_13"].sum()) + ", from $" + str(donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_13"].sum()) + " to $" + str(donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()) + " between 2013 and 2017.")
print("These donors' giving in the last two months of the year increased by $" + str(donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_11"].sum()) + ", from $" + str(donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_11"].sum()) + " to $" + str(donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()) + " between 2011 and 2017.")
print("These donors' giving in the last two months of the year increased by $" + str(donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_09"].sum()) + ", from $" + str(donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_09"].sum()) + " to $" + str(donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()) + " between 2009 and 2017.")

These donors' giving in the last two months of the year increased by $34550627.0, from $-6538636.0 to $28011991.0 between 2015 and 2017.
These donors' giving in the last two months of the year increased by $20592916.0, from $1123575.0 to $21716491.0 between 2013 and 2017.
These donors' giving in the last two months of the year increased by $23766549.0, from $652592.0 to $24419141.0 between 2011 and 2017.
These donors' giving in the last two months of the year increased by $27013891.0, from $365850.0 to $27379741.0 between 2009 and 2017.


## How many donors in each year gave all of their money in the last two months of each year?

In [54]:
print("In 2017, " + str(donors_17[donors_17["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")
print("In 2015, " + str(donors_15[donors_15["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")
print("In 2013, " + str(donors_13[donors_13["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")
print("In 2011, " + str(donors_11[donors_11["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")
print("In 2009, " + str(donors_09[donors_09["pct_post_bill"] == 1]["match_id"].count()) + " donors gave all their money in the last two months.")

In 2017, 39 donors gave all their money in the last two months.
In 2015, 2 donors gave all their money in the last two months.
In 2013, 3 donors gave all their money in the last two months.
In 2011, 3 donors gave all their money in the last two months.
In 2009, 6 donors gave all their money in the last two months.


## Return the donors who gave at least $50,000 to conservative and GOP-aligned groups between Feb. 2 and March 31, 2017, split by their giving inside and outside of that period. Do this for every off-year going back to the 2010 cycle.

In [55]:
def run_donors_query_1st_qtr(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start_one, end_one, start_two, end_two):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_donors_committees_""" + suffix + """ AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN left(contribid, 11) IN ('d1007764625',
                                              'q0001024006',
                                              'U0000000002',
                                              'U0000000003',
                                              'U0000004292',
                                              'U0000004308',
                                              'U0000004652') THEN 'DEVOS'
                 WHEN left(contribid, 11) IN ('U0000003449',
                                              'U0000003450') THEN 'FERTITTA'
                 WHEN left(contribid, 11) IN ('U0000003477',
                                              'U0000003492') THEN 'KOCH'
                 WHEN left(contribid, 11) IN ('h3001352487',
                                              'm0001331786') THEN 'HILDEBRAND'
                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '""" + reference_start + """'
        AND date <= '""" + reference_end + """'
        AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN left(contribid, 11) IN ('d1007764625',
                                                              'q0001024006',
                                                              'U0000000002',
                                                              'U0000000003',
                                                              'U0000004292',
                                                              'U0000004308',
                                                              'U0000004652') THEN 'DEVOS'
                                 WHEN left(contribid, 11) IN ('U0000003449',
                                                              'U0000003450') THEN 'FERTITTA'
                                 WHEN left(contribid, 11) IN ('U0000003477',
                                                              'U0000003492') THEN 'KOCH'
                                 WHEN left(contribid, 11) IN ('h3001352487',
                                                              'm0001331786') THEN 'HILDEBRAND'
                                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE orgname
                             END = gop_donors.match_id
   WHERE date >= '""" + start_one + """'
     AND date <= '""" + end_one + """'
     OR (date >= '""" + start_two + """'
         AND date <= '""" + end_two + """')
     AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '""" + cycle + """'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
  AND match_id NOT IN ('Air Line Pilots Assn',
                       'America First Policies',
                       'American Action Network',
                       'a0000720833',
                       'q0000962134',
                       'Republican Governors Assn',
                       'Southeast Wood Treating')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;


SELECT *
FROM """ + prefix + """_donors_committees_""" + suffix + """;"""
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [56]:
inside_bill_donors_committees_17_1st_qtr = run_donors_query_1st_qtr("inside", "17_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2018", "2017-02-02", "2017-03-31", "2017-02-02", "2017-03-31")
outside_bill_donors_committees_17_1st_qtr = run_donors_query_1st_qtr("outside", "17_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2018", "2017-01-01", "2017-02-01", "2017-04-01", "2017-12-31")

In [57]:
inside_bill_donors_committees_15_1st_qtr = run_donors_query_1st_qtr("inside", "15_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2016", "2015-02-02", "2015-03-31", "2015-02-02", "2015-03-31")
outside_bill_donors_committees_15_1st_qtr = run_donors_query_1st_qtr("outside", "15_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2016", "2015-01-01", "2015-02-01", "2015-04-01", "2015-12-31")

In [58]:
inside_bill_donors_committees_13_1st_qtr = run_donors_query_1st_qtr("inside", "13_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2014", "2013-02-02", "2013-03-31", "2013-02-02", "2013-03-31")
outside_bill_donors_committees_13_1st_qtr = run_donors_query_1st_qtr("outside", "13_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2014", "2013-01-01", "2013-02-01", "2013-04-01", "2013-12-31")

In [59]:
inside_bill_donors_committees_11_1st_qtr = run_donors_query_1st_qtr("inside", "11_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2012", "2011-02-02", "2011-03-31", "2011-02-02", "2011-03-31")
outside_bill_donors_committees_11_1st_qtr = run_donors_query_1st_qtr("outside", "11_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2012", "2011-01-01", "2011-02-01", "2011-04-01", "2011-12-31")

In [60]:
inside_bill_donors_committees_09_1st_qtr = run_donors_query_1st_qtr("inside", "09_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2010", "2009-02-02", "2009-03-31", "2009-02-02", "2009-03-31")
outside_bill_donors_committees_09_1st_qtr = run_donors_query_1st_qtr("outside", "09_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2010", "2009-01-01", "2009-02-01", "2009-04-01", "2009-12-31")

## How much did each contributor give in each period for each cycle?

### 2017

In [61]:
inside_bill_donors_17_1st_qtr = inside_bill_donors_committees_17_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_17_1st_qtr = outside_bill_donors_committees_17_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [62]:
donors_17_1st_qtr = inside_bill_donors_17_1st_qtr.merge(outside_bill_donors_17_1st_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_17_1st_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_17_1st_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_17_1st_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_17_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267 entries, 0 to 266
Data columns (total 5 columns):
match_id              267 non-null object
organizations         267 non-null object
contributors          267 non-null object
total_inside_bill     267 non-null float64
total_outside_bill    267 non-null float64
dtypes: float64(2), object(3)
memory usage: 12.5+ KB


In [63]:
donors_17_1st_qtr["pct_inside_bill"] = donors_17_1st_qtr["total_inside_bill"] / (donors_17_1st_qtr["total_outside_bill"] + donors_17_1st_qtr["total_inside_bill"])
donors_17_1st_qtr["pct_outside_bill"] = donors_17_1st_qtr["total_outside_bill"] / (donors_17_1st_qtr["total_outside_bill"] + donors_17_1st_qtr["total_inside_bill"])
donors_17_1st_qtr["change"] = donors_17_1st_qtr["total_inside_bill"] - donors_17_1st_qtr["total_outside_bill"]
donors_17_1st_qtr["pct_change"] = (donors_17_1st_qtr["total_inside_bill"] - donors_17_1st_qtr["total_outside_bill"]) / donors_17_1st_qtr["total_outside_bill"].abs()
donors_17_1st_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
68,U0000003690,[Uline Inc],"[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH A, UIH...","2,076,600.00","14,552,500.00",0.12,0.88,"-12,475,900.00",-0.86
1,Ai Altep Holdings,[Ai Altep Holdings],[AI ALTEP HOLDINGS INC],"1,000,000.00",0.00,1.00,0.00,"1,000,000.00",inf
39,U0000003265,"[Dimensional Fund Advisors, Show Me Institute]","[SINQUEFIELD, JEANNE C MRS, SINQUEFIELD, REX A]","968,600.00","99,800.00",0.91,0.09,"868,800.00",8.71
3,Chevron Corp,[Chevron Corp],"[CHEVRON, CHEVRON POLICY GOVERNMENT & PUBLIC A...","950,000.00","750,000.00",0.56,0.44,"200,000.00",0.27
49,U0000003462,[Anschutz Corp],"[ANSCHUTZ, NANCY P, ANSCHUTZ, NANCY PARKER, AN...","837,600.00","44,100.00",0.95,0.05,"793,500.00",17.99


#### How much did they give?

In [64]:
print("2017 inside-bill 1st quarter total: $" + str(donors_17_1st_qtr["total_inside_bill"].sum()))
print("2017 outside-bill 1st quarter total: $" + str(donors_17_1st_qtr["total_outside_bill"].sum()))

2017 inside-bill 1st quarter total: $49069603.0
2017 outside-bill 1st quarter total: $59662630.0


### 2015

In [65]:
inside_bill_donors_15_1st_qtr = inside_bill_donors_committees_15_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_15_1st_qtr = outside_bill_donors_committees_15_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [66]:
donors_15_1st_qtr = inside_bill_donors_15_1st_qtr.merge(outside_bill_donors_15_1st_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_15_1st_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_15_1st_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_15_1st_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_15_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 243 entries, 0 to 242
Data columns (total 5 columns):
match_id              243 non-null object
organizations         191 non-null object
contributors          191 non-null object
total_inside_bill     191 non-null float64
total_outside_bill    243 non-null float64
dtypes: float64(2), object(3)
memory usage: 11.4+ KB


In [67]:
donors_15_1st_qtr["pct_inside_bill"] = donors_15_1st_qtr["total_inside_bill"] / (donors_15_1st_qtr["total_outside_bill"] + donors_15_1st_qtr["total_inside_bill"])
donors_15_1st_qtr["pct_outside_bill"] = donors_15_1st_qtr["total_outside_bill"] / (donors_15_1st_qtr["total_outside_bill"] + donors_15_1st_qtr["total_inside_bill"])
donors_15_1st_qtr["change"] = donors_15_1st_qtr["total_inside_bill"] - donors_15_1st_qtr["total_outside_bill"]
donors_15_1st_qtr["pct_change"] = (donors_15_1st_qtr["total_inside_bill"] - donors_15_1st_qtr["total_outside_bill"]) / donors_15_1st_qtr["total_outside_bill"].abs()
donors_15_1st_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
19,U0000002708,[Mountaire Corp],"[CAMERON, RONALD, CAMERON, RONALD M MR]","3,012,700.00","191,500.00",0.94,0.06,"2,821,200.00",14.73
68,U0000004014,"[MBF Healthcare Partners, Mindful Kids Miami]","[FERNANDEZ, CONSTANCE, FERNANDEZ, CONSTANCE M,...","1,075,520.00","2,402,100.00",0.31,0.69,"-1,326,580.00",-0.55
53,U0000003658,"[Franklin Resources, Psychiatrist]","[JOHNSON, ANN L, JOHNSON, CHARLES, JOHNSON, CH...","1,029,600.00","663,900.00",0.61,0.39,"365,700.00",0.55
4,NextEra Energy,[NextEra Energy],[NEXTERA ENERGY],"1,025,000.00","250,000.00",0.80,0.20,"775,000.00",3.10
55,U0000003682,[Renaissance Technologies],"[MERCER, DIANA L MRS, MERCER, DIANA L MS, MERC...","754,600.00","14,323,000.00",0.05,0.95,"-13,568,400.00",-0.95


#### How much did they give?

In [68]:
print("2015 inside-bill 1st quarter total: $" + str(donors_15_1st_qtr["total_inside_bill"].sum()))
print("2015 outside-bill 1st quarter total: $" + str(donors_15_1st_qtr["total_outside_bill"].sum()))

2015 inside-bill 1st quarter total: $20592227.0
2015 outside-bill 1st quarter total: $103498756.0


### 2013

In [69]:
inside_bill_donors_13_1st_qtr = inside_bill_donors_committees_13_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_13_1st_qtr = outside_bill_donors_committees_13_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [70]:
donors_13_1st_qtr = inside_bill_donors_13_1st_qtr.merge(outside_bill_donors_13_1st_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_13_1st_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_13_1st_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_13_1st_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_13_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 0 to 211
Data columns (total 5 columns):
match_id              212 non-null object
organizations         131 non-null object
contributors          131 non-null object
total_inside_bill     131 non-null float64
total_outside_bill    212 non-null float64
dtypes: float64(2), object(3)
memory usage: 9.9+ KB


In [71]:
donors_13_1st_qtr["pct_inside_bill"] = donors_13_1st_qtr["total_inside_bill"] / (donors_13_1st_qtr["total_outside_bill"] + donors_13_1st_qtr["total_inside_bill"])
donors_13_1st_qtr["pct_outside_bill"] = donors_13_1st_qtr["total_outside_bill"] / (donors_13_1st_qtr["total_outside_bill"] + donors_13_1st_qtr["total_inside_bill"])
donors_13_1st_qtr["change"] = donors_13_1st_qtr["total_inside_bill"] - donors_13_1st_qtr["total_outside_bill"]
donors_13_1st_qtr["pct_change"] = (donors_13_1st_qtr["total_inside_bill"] - donors_13_1st_qtr["total_outside_bill"]) / donors_13_1st_qtr["total_outside_bill"].abs()
donors_13_1st_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
61,U0000004145,[Tommar LLC],"[LARKIN JR, THOMAS E, LARKIN, MARGARET, LARKIN...","210,400.00","40,600.00",0.84,0.16,"169,800.00",4.18
46,U0000003690,[Uline Inc],"[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH A, UIH...","203,700.00","590,850.00",0.26,0.74,"-387,150.00",-0.66
33,U0000003558,[Satter Investment Management],"[SATTER, MUNEER A]","175,200.00","187,800.00",0.48,0.52,"-12,600.00",-0.07
64,U0000004227,"[Kinder Foundation, Kinder Morgan Inc]","[KINDER, NANCY G, KINDER, NANCY G MR, KINDER, ...","165,900.00","10,000.00",0.94,0.06,"155,900.00",15.59
44,U0000003658,"[Franklin Resources, Psychiatrist]","[JOHNSON, ANN L, JOHNSON, CHARLES, JOHNSON, CH...","155,200.00","11,066.00",0.93,0.07,"144,134.00",13.02


#### How much did they give?

In [72]:
print("2013 inside-bill 1st quarter total: $" + str(donors_13_1st_qtr["total_inside_bill"].sum()))
print("2013 outside-bill 1st quarter total: $" + str(donors_13_1st_qtr["total_outside_bill"].sum()))

2013 inside-bill 1st quarter total: $4389003.0
2013 outside-bill 1st quarter total: $15929353.0


### 2011

In [73]:
inside_bill_donors_11_1st_qtr = inside_bill_donors_committees_11_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_11_1st_qtr = outside_bill_donors_committees_11_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [74]:
donors_11_1st_qtr = inside_bill_donors_11_1st_qtr.merge(outside_bill_donors_11_1st_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_11_1st_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_11_1st_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_11_1st_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_11_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 215 entries, 0 to 214
Data columns (total 5 columns):
match_id              215 non-null object
organizations         129 non-null object
contributors          129 non-null object
total_inside_bill     129 non-null float64
total_outside_bill    215 non-null float64
dtypes: float64(2), object(3)
memory usage: 10.1+ KB


In [75]:
donors_11_1st_qtr["pct_inside_bill"] = donors_11_1st_qtr["total_inside_bill"] / (donors_11_1st_qtr["total_outside_bill"] + donors_11_1st_qtr["total_inside_bill"])
donors_11_1st_qtr["pct_outside_bill"] = donors_11_1st_qtr["total_outside_bill"] / (donors_11_1st_qtr["total_outside_bill"] + donors_11_1st_qtr["total_inside_bill"])
donors_11_1st_qtr["change"] = donors_11_1st_qtr["total_inside_bill"] - donors_11_1st_qtr["total_outside_bill"]
donors_11_1st_qtr["pct_change"] = (donors_11_1st_qtr["total_inside_bill"] - donors_11_1st_qtr["total_outside_bill"]) / donors_11_1st_qtr["total_outside_bill"].abs()
donors_11_1st_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
53,U0000003932,"[[24I Contribution], Research Affiliates LLC]","[ARNOTT, ROBERT D, ARNOTT, ROBERT D MR]","267,600.00","312,500.00",0.46,0.54,"-44,900.00",-0.14
33,U0000003558,[Satter Investment Management],"[SATTER, MUNEER A]","150,000.00","289,000.00",0.34,0.66,"-139,000.00",-0.48
37,U0000003614,[PVS Chemicals],"[NICHOLSON, JAMES B]","117,000.00","69,000.00",0.63,0.37,"48,000.00",0.70
38,U0000003616,[Centra Inc],"[MOROUN, MATTHEW, MOROUN, MATTHEW T MR]","92,600.00","71,300.00",0.56,0.44,"21,300.00",0.30
18,U0000003246,[Jim Click Automotive],"[CLICK, JIM, CLICK, JIM JR, CLICK, VICKI, CLIC...","74,100.00","93,300.00",0.44,0.56,"-19,200.00",-0.21


#### How much did they give?

In [76]:
print("2011 inside-bill 1st quarter total: $" + str(donors_11_1st_qtr["total_inside_bill"].sum()))
print("2011 outside-bill 1st quarter total: $" + str(donors_11_1st_qtr["total_outside_bill"].sum()))

2011 inside-bill 1st quarter total: $3032290.0
2011 outside-bill 1st quarter total: $18826459.0


### 2009

In [77]:
inside_bill_donors_09_1st_qtr = inside_bill_donors_committees_09_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_09_1st_qtr = outside_bill_donors_committees_09_1st_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [78]:
donors_09_1st_qtr = inside_bill_donors_09_1st_qtr.merge(outside_bill_donors_09_1st_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_09_1st_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_09_1st_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_09_1st_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_09_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 5 columns):
match_id              180 non-null object
organizations         107 non-null object
contributors          107 non-null object
total_inside_bill     107 non-null float64
total_outside_bill    180 non-null float64
dtypes: float64(2), object(3)
memory usage: 8.4+ KB


In [79]:
donors_09_1st_qtr["pct_inside_bill"] = donors_09_1st_qtr["total_inside_bill"] / (donors_09_1st_qtr["total_outside_bill"] + donors_09_1st_qtr["total_inside_bill"])
donors_09_1st_qtr["pct_outside_bill"] = donors_09_1st_qtr["total_outside_bill"] / (donors_09_1st_qtr["total_outside_bill"] + donors_09_1st_qtr["total_inside_bill"])
donors_09_1st_qtr["change"] = donors_09_1st_qtr["total_inside_bill"] - donors_09_1st_qtr["total_outside_bill"]
donors_09_1st_qtr["pct_change"] = (donors_09_1st_qtr["total_inside_bill"] - donors_09_1st_qtr["total_outside_bill"]) / donors_09_1st_qtr["total_outside_bill"].abs()
donors_09_1st_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
22,U0000003295,"[[24T Contribution], Richard & Barbara Gaby Fo...","[GABY, BARBARA, GABY, RICHARD, GABY, RICHARD D...","90,000.00","62,800.00",0.59,0.41,"27,200.00",0.43
8,U0000000280,[Crow Holdings],"[CROW, HARLAN, CROW, HARLAN MR, CROW, HARLAN R...","80,600.00","176,800.00",0.31,0.69,"-96,200.00",-0.54
16,U0000003246,[Jim Click Automotive],"[CLICK, JIM, CLICK, JIM JR, CLICK, VICKI, CLIC...","65,800.00","19,400.00",0.77,0.23,"46,400.00",2.39
20,U0000003292,[Cintas Corp],"[FARMER, JOYCE E, FARMER, RICHARD T, FARMER, R T]","62,300.00","88,000.00",0.41,0.59,"-25,700.00",-0.29
21,U0000003293,[Sable Minerals],"[FLORES, CHERIE H, FLORES, CHERIE H MRS, FLORE...","60,800.00","18,700.00",0.76,0.24,"42,100.00",2.25


In [80]:
print("2009 inside-bill 1st quarter total: $" + str(donors_09_1st_qtr["total_inside_bill"].sum()))
print("2009 outside-bill 1st quarter total: $" + str(donors_09_1st_qtr["total_outside_bill"].sum()))

2009 inside-bill 1st quarter total: $1403605.0
2009 outside-bill 1st quarter total: $5133871.0


## How did each contributor's giving pattern change in each cycle compared with 2017?

### 2015

In [81]:
donors_15_17_1st_qtr = donors_17_1st_qtr.merge(donors_15_1st_qtr, how="outer", on="match_id", suffixes=["_17", "_15"])
donors_15_17_1st_qtr.drop(["change_17", "pct_change_17", "change_15", "pct_change_15"],
            axis=1, inplace=True)
donors_15_17_1st_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_15_17_1st_qtr = donors_15_17_1st_qtr[["match_id", "contributors", "organizations", "total_outside_bill_15", "total_inside_bill_15",
                 "pct_outside_bill_15", "pct_inside_bill_15", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_15_17_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267 entries, 0 to 266
Data columns (total 11 columns):
match_id                 267 non-null object
contributors             267 non-null object
organizations            267 non-null object
total_outside_bill_15    243 non-null float64
total_inside_bill_15     191 non-null float64
pct_outside_bill_15      191 non-null float64
pct_inside_bill_15       191 non-null float64
total_outside_bill_17    267 non-null float64
total_inside_bill_17     267 non-null float64
pct_outside_bill_17      267 non-null float64
pct_inside_bill_17       267 non-null float64
dtypes: float64(8), object(3)
memory usage: 25.0+ KB


In [82]:
donors_15_17_1st_qtr["giving_change"] = np.where((donors_15_17_1st_qtr["pct_inside_bill_17"] > donors_15_17_1st_qtr["pct_inside_bill_15"]) | (donors_15_17_1st_qtr["pct_outside_bill_15"].isnull()), "Increased",
                                  np.where(donors_15_17_1st_qtr["pct_inside_bill_17"] < donors_15_17_1st_qtr["pct_inside_bill_15"], "Decreased",
                                           np.where(donors_15_17_1st_qtr["pct_inside_bill_17"] == donors_15_17_1st_qtr["pct_inside_bill_15"], "Stayed the same",
                                                   "Other")))
donors_15_17_1st_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_15,total_inside_bill_15,pct_outside_bill_15,pct_inside_bill_15,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Agua Caliente Band of Cahuilla Indians,"[AGUA CALIENTE BAND OF CAHUILLA, AGUA CALIENTE...",[Agua Caliente Band of Cahuilla Indians],"83,500.00","50,400.00",0.62,0.38,"198,300.00","57,900.00",0.77,0.23,Decreased


### 2013

In [83]:
donors_13_17_1st_qtr = donors_17_1st_qtr.merge(donors_13_1st_qtr, how="outer", on="match_id", suffixes=["_17", "_13"])
donors_13_17_1st_qtr.drop(["change_17", "pct_change_17", "change_13", "pct_change_13"],
            axis=1, inplace=True)
donors_13_17_1st_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_13_17_1st_qtr = donors_13_17_1st_qtr[["match_id", "contributors", "organizations", "total_outside_bill_13", "total_inside_bill_13",
                 "pct_outside_bill_13", "pct_inside_bill_13", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_13_17_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267 entries, 0 to 266
Data columns (total 11 columns):
match_id                 267 non-null object
contributors             267 non-null object
organizations            267 non-null object
total_outside_bill_13    212 non-null float64
total_inside_bill_13     131 non-null float64
pct_outside_bill_13      131 non-null float64
pct_inside_bill_13       131 non-null float64
total_outside_bill_17    267 non-null float64
total_inside_bill_17     267 non-null float64
pct_outside_bill_17      267 non-null float64
pct_inside_bill_17       267 non-null float64
dtypes: float64(8), object(3)
memory usage: 25.0+ KB


In [84]:
donors_13_17_1st_qtr["giving_change"] = np.where((donors_13_17_1st_qtr["pct_inside_bill_17"] > donors_13_17_1st_qtr["pct_inside_bill_13"]) | (donors_13_17_1st_qtr["pct_outside_bill_13"].isnull()), "Increased",
                                  np.where(donors_13_17_1st_qtr["pct_inside_bill_17"] < donors_13_17_1st_qtr["pct_inside_bill_13"], "Decreased",
                                           np.where(donors_13_17_1st_qtr["pct_inside_bill_17"] == donors_13_17_1st_qtr["pct_inside_bill_13"], "Stayed the same",
                                                   "Other")))
donors_13_17_1st_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_13,total_inside_bill_13,pct_outside_bill_13,pct_inside_bill_13,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Agua Caliente Band of Cahuilla Indians,"[AGUA CALIENTE BAND OF CAHUILLA, AGUA CALIENTE...",[Agua Caliente Band of Cahuilla Indians],"29,450.00","77,600.00",0.28,0.72,"198,300.00","57,900.00",0.77,0.23,Decreased


### 2011

In [85]:
donors_11_17_1st_qtr = donors_17_1st_qtr.merge(donors_11_1st_qtr, how="outer", on="match_id", suffixes=["_17", "_11"])
donors_11_17_1st_qtr.drop(["change_17", "pct_change_17", "change_11", "pct_change_11"],
            axis=1, inplace=True)
donors_11_17_1st_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_11_17_1st_qtr = donors_11_17_1st_qtr[["match_id", "contributors", "organizations", "total_outside_bill_11", "total_inside_bill_11",
                 "pct_outside_bill_11", "pct_inside_bill_11", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_11_17_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267 entries, 0 to 266
Data columns (total 11 columns):
match_id                 267 non-null object
contributors             267 non-null object
organizations            267 non-null object
total_outside_bill_11    215 non-null float64
total_inside_bill_11     129 non-null float64
pct_outside_bill_11      129 non-null float64
pct_inside_bill_11       129 non-null float64
total_outside_bill_17    267 non-null float64
total_inside_bill_17     267 non-null float64
pct_outside_bill_17      267 non-null float64
pct_inside_bill_17       267 non-null float64
dtypes: float64(8), object(3)
memory usage: 25.0+ KB


In [86]:
donors_11_17_1st_qtr["giving_change"] = np.where((donors_11_17_1st_qtr["pct_inside_bill_17"] > donors_11_17_1st_qtr["pct_inside_bill_11"]) | (donors_11_17_1st_qtr["pct_outside_bill_11"].isnull()), "Increased",
                                  np.where(donors_11_17_1st_qtr["pct_inside_bill_17"] < donors_11_17_1st_qtr["pct_inside_bill_11"], "Decreased",
                                           np.where(donors_11_17_1st_qtr["pct_inside_bill_17"] == donors_11_17_1st_qtr["pct_inside_bill_11"], "Stayed the same",
                                                   "Other")))
donors_11_17_1st_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_11,total_inside_bill_11,pct_outside_bill_11,pct_inside_bill_11,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Agua Caliente Band of Cahuilla Indians,"[AGUA CALIENTE BAND OF CAHUILLA, AGUA CALIENTE...",[Agua Caliente Band of Cahuilla Indians],"53,800.00","2,000.00",0.96,0.04,"198,300.00","57,900.00",0.77,0.23,Increased


### 2009

In [87]:
donors_09_17_1st_qtr = donors_17_1st_qtr.merge(donors_09_1st_qtr, how="outer", on="match_id", suffixes=["_17", "_09"])
donors_09_17_1st_qtr.drop(["change_17", "pct_change_17", "change_09", "pct_change_09"],
            axis=1, inplace=True)
donors_09_17_1st_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_09_17_1st_qtr = donors_09_17_1st_qtr[["match_id", "contributors", "organizations", "total_outside_bill_09", "total_inside_bill_09",
                 "pct_outside_bill_09", "pct_inside_bill_09", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_09_17_1st_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267 entries, 0 to 266
Data columns (total 11 columns):
match_id                 267 non-null object
contributors             267 non-null object
organizations            267 non-null object
total_outside_bill_09    180 non-null float64
total_inside_bill_09     107 non-null float64
pct_outside_bill_09      107 non-null float64
pct_inside_bill_09       107 non-null float64
total_outside_bill_17    267 non-null float64
total_inside_bill_17     267 non-null float64
pct_outside_bill_17      267 non-null float64
pct_inside_bill_17       267 non-null float64
dtypes: float64(8), object(3)
memory usage: 25.0+ KB


In [88]:
donors_09_17_1st_qtr["giving_change"] = np.where((donors_09_17_1st_qtr["pct_inside_bill_17"] > donors_09_17_1st_qtr["pct_inside_bill_09"]) | (donors_09_17_1st_qtr["pct_outside_bill_09"].isnull()), "Increased",
                                  np.where(donors_09_17_1st_qtr["pct_inside_bill_17"] < donors_09_17_1st_qtr["pct_inside_bill_09"], "Decreased",
                                           np.where(donors_09_17_1st_qtr["pct_inside_bill_17"] == donors_09_17_1st_qtr["pct_inside_bill_09"], "Stayed the same",
                                                   "Other")))
donors_09_17_1st_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_09,total_inside_bill_09,pct_outside_bill_09,pct_inside_bill_09,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Agua Caliente Band of Cahuilla Indians,"[AGUA CALIENTE BAND OF CAHUILLA, AGUA CALIENTE...",[Agua Caliente Band of Cahuilla Indians],"6,000.00","30,400.00",0.16,0.84,"198,300.00","57,900.00",0.77,0.23,Decreased


## What proportion of donors increased the share of their annual giving that fell within the two-month period in 2017 as compared with the same period in each of the previous years?

In [89]:
print(str(donors_15_17_1st_qtr[donors_15_17_1st_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_15_17_1st_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2015 and 2017.")
print(str(donors_13_17_1st_qtr[donors_13_17_1st_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_13_17_1st_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2013 and 2017.")
print(str(donors_11_17_1st_qtr[donors_11_17_1st_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_11_17_1st_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2011 and 2017.")
print(str(donors_09_17_1st_qtr[donors_09_17_1st_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_09_17_1st_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2009 and 2017.")

87.26591760299625% of donors increased the share of their annual giving that fell within the two-month period between 2015 and 2017.
88.01498127340824% of donors increased the share of their annual giving that fell within the two-month period between 2013 and 2017.
94.3820224719101% of donors increased the share of their annual giving that fell within the two-month period between 2011 and 2017.
89.51310861423221% of donors increased the share of their annual giving that fell within the two-month period between 2009 and 2017.


## And by how much did these individuals' giving increase?

In [90]:
print("These donors' giving within the two-month period increased by $" + str(donors_15_17_1st_qtr[donors_15_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_15_17_1st_qtr[donors_15_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_15"].sum()) + ", from $" + str(donors_15_17_1st_qtr[donors_15_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_15"].sum()) + " to $" + str(donors_15_17_1st_qtr[donors_15_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2015 and 2017.")
print("These donors' giving within the two-month period increased by $" + str(donors_13_17_1st_qtr[donors_13_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_13_17_1st_qtr[donors_13_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_13"].sum()) + ", from $" + str(donors_13_17_1st_qtr[donors_13_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_13"].sum()) + " to $" + str(donors_13_17_1st_qtr[donors_13_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2013 and 2017.")
print("These donors' giving within the two-month period increased by $" + str(donors_11_17_1st_qtr[donors_11_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_11_17_1st_qtr[donors_11_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_11"].sum()) + ", from $" + str(donors_11_17_1st_qtr[donors_11_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_11"].sum()) + " to $" + str(donors_11_17_1st_qtr[donors_11_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2011 and 2017.")
print("These donors' giving within the two-month period increased by $" + str(donors_09_17_1st_qtr[donors_09_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_09_17_1st_qtr[donors_09_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_09"].sum()) + ", from $" + str(donors_09_17_1st_qtr[donors_09_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_09"].sum()) + " to $" + str(donors_09_17_1st_qtr[donors_09_17_1st_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2009 and 2017.")

These donors' giving within the two-month period increased by $33014543.0, from $11774527.0 to $44789070.0 between 2015 and 2017.
These donors' giving within the two-month period increased by $39925669.0, from $2505309.0 to $42430978.0 between 2013 and 2017.
These donors' giving within the two-month period increased by $45438183.0, from $2286890.0 to $47725073.0 between 2011 and 2017.
These donors' giving within the two-month period increased by $43270415.0, from $744955.0 to $44015370.0 between 2009 and 2017.


## How many donors in each year gave all of their money in the two-month period of each year?

In [91]:
print("In 2017, " + str(donors_17_1st_qtr[donors_17_1st_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2015, " + str(donors_15_1st_qtr[donors_15_1st_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2013, " + str(donors_13_1st_qtr[donors_13_1st_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2011, " + str(donors_11_1st_qtr[donors_11_1st_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2009, " + str(donors_09_1st_qtr[donors_09_1st_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")

In 2017, 43 donors gave all their money in the two-month period.
In 2015, 6 donors gave all their money in the two-month period.
In 2013, 5 donors gave all their money in the two-month period.
In 2011, 1 donors gave all their money in the two-month period.
In 2009, 8 donors gave all their money in the two-month period.


## Return the donors who gave at least $50,000 to conservative and GOP-aligned groups between May 2 and June 30, 2017, split by their giving inside and outside of that period. Do this for every off-year going back to the 2010 cycle.

In [92]:
def run_donors_query_2nd_qtr(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start_one, end_one, start_two, end_two):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_donors_committees_""" + suffix + """ AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN left(contribid, 11) IN ('d1007764625',
                                              'q0001024006',
                                              'U0000000002',
                                              'U0000000003',
                                              'U0000004292',
                                              'U0000004308',
                                              'U0000004652') THEN 'DEVOS'
                 WHEN left(contribid, 11) IN ('U0000003449',
                                              'U0000003450') THEN 'FERTITTA'
                 WHEN left(contribid, 11) IN ('U0000003477',
                                              'U0000003492') THEN 'KOCH'
                 WHEN left(contribid, 11) IN ('h3001352487',
                                              'm0001331786') THEN 'HILDEBRAND'
                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '""" + reference_start + """'
        AND date <= '""" + reference_end + """'
        AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN left(contribid, 11) IN ('d1007764625',
                                                              'q0001024006',
                                                              'U0000000002',
                                                              'U0000000003',
                                                              'U0000004292',
                                                              'U0000004308',
                                                              'U0000004652') THEN 'DEVOS'
                                 WHEN left(contribid, 11) IN ('U0000003449',
                                                              'U0000003450') THEN 'FERTITTA'
                                 WHEN left(contribid, 11) IN ('U0000003477',
                                                              'U0000003492') THEN 'KOCH'
                                 WHEN left(contribid, 11) IN ('h3001352487',
                                                              'm0001331786') THEN 'HILDEBRAND'
                                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE orgname
                             END = gop_donors.match_id
   WHERE date >= '""" + start_one + """'
     AND date <= '""" + end_one + """'
     OR (date >= '""" + start_two + """'
         AND date <= '""" + end_two + """')
     AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '""" + cycle + """'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
  AND match_id NOT IN ('Air Line Pilots Assn',
                       'America First Policies',
                       'American Action Network',
                       'a0000720833',
                       'q0000962134',
                       'Republican Governors Assn',
                       'Southeast Wood Treating')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;


SELECT *
FROM """ + prefix + """_donors_committees_""" + suffix + """;"""
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [93]:
inside_bill_donors_committees_17_2nd_qtr = run_donors_query_2nd_qtr("inside", "17_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2018", "2017-05-02", "2017-06-30", "2017-05-02", "2017-06-30")
outside_bill_donors_committees_17_2nd_qtr = run_donors_query_2nd_qtr("outside", "17_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2018", "2017-01-01", "2017-05-01", "2017-07-01", "2017-12-31")

In [94]:
inside_bill_donors_committees_15_2nd_qtr = run_donors_query_2nd_qtr("inside", "15_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2016", "2015-05-02", "2015-06-30", "2015-05-02", "2015-06-30")
outside_bill_donors_committees_15_2nd_qtr = run_donors_query_2nd_qtr("outside", "15_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2016", "2015-01-01", "2015-05-01", "2015-07-01", "2015-12-31")

In [95]:
inside_bill_donors_committees_13_2nd_qtr = run_donors_query_2nd_qtr("inside", "13_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2014", "2013-05-02", "2013-06-30", "2013-05-02", "2013-06-30")
outside_bill_donors_committees_13_2nd_qtr = run_donors_query_2nd_qtr("outside", "13_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2014", "2013-01-01", "2013-05-01", "2013-07-01", "2013-12-31")

In [96]:
inside_bill_donors_committees_11_2nd_qtr = run_donors_query_2nd_qtr("inside", "11_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2012", "2011-05-02", "2011-06-30", "2011-05-02", "2011-06-30")
outside_bill_donors_committees_11_2nd_qtr = run_donors_query_2nd_qtr("outside", "11_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2012", "2011-01-01", "2011-05-01", "2011-07-01", "2011-12-31")

In [97]:
inside_bill_donors_committees_09_2nd_qtr = run_donors_query_2nd_qtr("inside", "09_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2010", "2009-05-02", "2009-06-30", "2009-05-02", "2009-06-30")
outside_bill_donors_committees_09_2nd_qtr = run_donors_query_2nd_qtr("outside", "09_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2010", "2009-01-01", "2009-05-01", "2009-07-01", "2009-12-31")

## How much did each contributor give in each period for each cycle?

### 2017

In [98]:
inside_bill_donors_17_2nd_qtr = inside_bill_donors_committees_17_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_17_2nd_qtr = outside_bill_donors_committees_17_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [99]:
donors_17_2nd_qtr = inside_bill_donors_17_2nd_qtr.merge(outside_bill_donors_17_2nd_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_17_2nd_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_17_2nd_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_17_2nd_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_17_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 5 columns):
match_id              217 non-null object
organizations         217 non-null object
contributors          217 non-null object
total_inside_bill     217 non-null float64
total_outside_bill    217 non-null float64
dtypes: float64(2), object(3)
memory usage: 10.2+ KB


In [100]:
donors_17_2nd_qtr["pct_inside_bill"] = donors_17_2nd_qtr["total_inside_bill"] / (donors_17_2nd_qtr["total_outside_bill"] + donors_17_2nd_qtr["total_inside_bill"])
donors_17_2nd_qtr["pct_outside_bill"] = donors_17_2nd_qtr["total_outside_bill"] / (donors_17_2nd_qtr["total_outside_bill"] + donors_17_2nd_qtr["total_inside_bill"])
donors_17_2nd_qtr["change"] = donors_17_2nd_qtr["total_inside_bill"] - donors_17_2nd_qtr["total_outside_bill"]
donors_17_2nd_qtr["pct_change"] = (donors_17_2nd_qtr["total_inside_bill"] - donors_17_2nd_qtr["total_outside_bill"]) / donors_17_2nd_qtr["total_outside_bill"].abs()
donors_17_2nd_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
52,U0000003690,"[[24T Contribution], Uline Inc, None]","[UIHLEIN, RICHARD, UIHLEIN, RICHARD E, UIHLEIN...","3,263,500.00","13,454,300.00",0.20,0.80,"-10,190,800.00",-0.76
18,U0000000073,[Western Refining],"[FOSTER, PAUL L]","1,495,400.00","308,400.00",0.83,0.17,"1,187,000.00",3.85
7,FERTITTA,[Station Casinos],"[FERTITTA, FRANK J III, FERTITTA, FRANK J MR I...","1,400,000.00","1,356,100.00",0.51,0.49,"43,900.00",0.03
116,d0000350730,[Aileron],"[MATHILE, CLAYTON]","1,000,000.00",0.00,1.00,0.00,"1,000,000.00",inf
51,U0000003682,[Renaissance Technologies],"[MERCER, DIANA L MS, MERCER, ROBERT, MERCER, R...","868,000.00","3,297,100.00",0.21,0.79,"-2,429,100.00",-0.74


#### How much did they give?

In [101]:
print("2017 inside-bill 2nd quarter total: $" + str(donors_17_2nd_qtr["total_inside_bill"].sum()))
print("2017 outside-bill 2nd quarter total: $" + str(donors_17_2nd_qtr["total_outside_bill"].sum()))

2017 inside-bill 2nd quarter total: $43044683.0
2017 outside-bill 2nd quarter total: $58691285.0


### 2015

In [102]:
inside_bill_donors_15_2nd_qtr = inside_bill_donors_committees_15_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_15_2nd_qtr = outside_bill_donors_committees_15_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [103]:
donors_15_2nd_qtr = inside_bill_donors_15_2nd_qtr.merge(outside_bill_donors_15_2nd_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_15_2nd_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_15_2nd_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_15_2nd_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_15_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 184 entries, 0 to 183
Data columns (total 5 columns):
match_id              184 non-null object
organizations         147 non-null object
contributors          147 non-null object
total_inside_bill     147 non-null float64
total_outside_bill    184 non-null float64
dtypes: float64(2), object(3)
memory usage: 8.6+ KB


In [104]:
donors_15_2nd_qtr["pct_inside_bill"] = donors_15_2nd_qtr["total_inside_bill"] / (donors_15_2nd_qtr["total_outside_bill"] + donors_15_2nd_qtr["total_inside_bill"])
donors_15_2nd_qtr["pct_outside_bill"] = donors_15_2nd_qtr["total_outside_bill"] / (donors_15_2nd_qtr["total_outside_bill"] + donors_15_2nd_qtr["total_inside_bill"])
donors_15_2nd_qtr["change"] = donors_15_2nd_qtr["total_inside_bill"] - donors_15_2nd_qtr["total_outside_bill"]
donors_15_2nd_qtr["pct_change"] = (donors_15_2nd_qtr["total_inside_bill"] - donors_15_2nd_qtr["total_outside_bill"]) / donors_15_2nd_qtr["total_outside_bill"].abs()
donors_15_2nd_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
8,U0000000074,[ABC Supply],"[HENDRICKS, DIANE, HENDRICKS, DIANE M MS]","5,000,000.00","-1,712,400.00",1.52,-0.52,"6,712,400.00",3.92
41,U0000003690,"[[24T Contribution], Uline Inc, None]","[UIHLEIN, RICHARD, UIHLEIN, RICHARD E, UIHLEIN...","3,196,100.00","2,143,200.00",0.60,0.40,"1,052,900.00",0.49
25,U0000003307,"[Bernard Marcus Family Foundation, None]","[MARCUS, BERNARD, MARCUS, BERNARD MR, MARCUS, ...","2,489,500.00","871,475.00",0.74,0.26,"1,618,025.00",1.86
57,U0000004245,[Susquehanna International Group],"[YASS, JANINE, YASS, JANINE MRS, YASS, JEFF, Y...","2,282,400.00","768,900.00",0.75,0.25,"1,513,500.00",1.97
6,U0000000066,[Elliott Management],"[SINGER, PAUL, SINGER, PAUL E MR, SINGER, PAUL...","1,681,590.00","4,904,992.00",0.26,0.74,"-3,223,402.00",-0.66


#### How much did they give?

In [105]:
print("2015 inside-bill 2nd quarter total: $" + str(donors_15_2nd_qtr["total_inside_bill"].sum()))
print("2015 outside-bill 2nd quarter total: $" + str(donors_15_2nd_qtr["total_outside_bill"].sum()))

2015 inside-bill 2nd quarter total: $33068246.0
2015 outside-bill 2nd quarter total: $74048521.0


### 2013

In [106]:
inside_bill_donors_13_2nd_qtr = inside_bill_donors_committees_13_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_13_2nd_qtr = outside_bill_donors_committees_13_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [107]:
donors_13_2nd_qtr = inside_bill_donors_13_2nd_qtr.merge(outside_bill_donors_13_2nd_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_13_2nd_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_13_2nd_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_13_2nd_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_13_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156 entries, 0 to 155
Data columns (total 5 columns):
match_id              156 non-null object
organizations         113 non-null object
contributors          113 non-null object
total_inside_bill     113 non-null float64
total_outside_bill    156 non-null float64
dtypes: float64(2), object(3)
memory usage: 7.3+ KB


In [108]:
donors_13_2nd_qtr["pct_inside_bill"] = donors_13_2nd_qtr["total_inside_bill"] / (donors_13_2nd_qtr["total_outside_bill"] + donors_13_2nd_qtr["total_inside_bill"])
donors_13_2nd_qtr["pct_outside_bill"] = donors_13_2nd_qtr["total_outside_bill"] / (donors_13_2nd_qtr["total_outside_bill"] + donors_13_2nd_qtr["total_inside_bill"])
donors_13_2nd_qtr["change"] = donors_13_2nd_qtr["total_inside_bill"] - donors_13_2nd_qtr["total_outside_bill"]
donors_13_2nd_qtr["pct_change"] = (donors_13_2nd_qtr["total_inside_bill"] - donors_13_2nd_qtr["total_outside_bill"]) / donors_13_2nd_qtr["total_outside_bill"].abs()
donors_13_2nd_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
50,U0000004245,[Susquehanna International Group],"[YASS, JANINE, YASS, JANINE MRS, YASS, JEFF, Y...","270,400.00","25,900.00",0.91,0.09,"244,500.00",9.44
35,U0000003682,[Renaissance Technologies],"[MERCER, DIANA L MS, MERCER, ROBERT, MERCER, R...","208,800.00","89,600.00",0.70,0.30,"119,200.00",1.33
44,U0000004017,"[Bluff Point Assoc, Rein Capitol LLC]","[MCINERNEY, PAULA, MCINERNEY, PAULA G MRS, MC ...","162,400.00","188,000.00",0.46,0.54,"-25,600.00",-0.14
30,U0000003614,[PVS Chemicals],"[NICHOLSON, ANN, NICHOLSON, ANN V, NICHOLSON, ...","123,300.00","78,800.00",0.61,0.39,"44,500.00",0.56
60,U0000004622,"[Ruffin Acquisitions LLC, Treasure Island]","[RUFFIN, PHILLIP, RUFFIN, PHILLIP MR]","116,070.00","9,600.00",0.92,0.08,"106,470.00",11.09


#### How much did they give?

In [109]:
print("2013 inside-bill 2nd quarter total: $" + str(donors_13_2nd_qtr["total_inside_bill"].sum()))
print("2013 outside-bill 2nd quarter total: $" + str(donors_13_2nd_qtr["total_outside_bill"].sum()))

2013 inside-bill 2nd quarter total: $3657157.0
2013 outside-bill 2nd quarter total: $11055381.0


### 2011

In [110]:
inside_bill_donors_11_2nd_qtr = inside_bill_donors_committees_11_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_11_2nd_qtr = outside_bill_donors_committees_11_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [111]:
donors_11_2nd_qtr = inside_bill_donors_11_2nd_qtr.merge(outside_bill_donors_11_2nd_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_11_2nd_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_11_2nd_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_11_2nd_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_11_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 170 entries, 0 to 169
Data columns (total 5 columns):
match_id              170 non-null object
organizations         116 non-null object
contributors          116 non-null object
total_inside_bill     116 non-null float64
total_outside_bill    170 non-null float64
dtypes: float64(2), object(3)
memory usage: 8.0+ KB


In [112]:
donors_11_2nd_qtr["pct_inside_bill"] = donors_11_2nd_qtr["total_inside_bill"] / (donors_11_2nd_qtr["total_outside_bill"] + donors_11_2nd_qtr["total_inside_bill"])
donors_11_2nd_qtr["pct_outside_bill"] = donors_11_2nd_qtr["total_outside_bill"] / (donors_11_2nd_qtr["total_outside_bill"] + donors_11_2nd_qtr["total_inside_bill"])
donors_11_2nd_qtr["change"] = donors_11_2nd_qtr["total_inside_bill"] - donors_11_2nd_qtr["total_outside_bill"]
donors_11_2nd_qtr["pct_change"] = (donors_11_2nd_qtr["total_inside_bill"] - donors_11_2nd_qtr["total_outside_bill"]) / donors_11_2nd_qtr["total_outside_bill"].abs()
donors_11_2nd_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
22,U0000003585,"[Homemaker, Putnam Investments, None]","[REYNOLDS, LAURA C, REYNOLDS, ROBERT, REYNOLDS...","236,000.00","64,700.00",0.78,0.22,"171,300.00",2.65
31,U0000003819,[Caxton Alternative Management],"[KOVNER, BRUCE, KOVNER, BRUCE MR, KOVNER, SUZA...","180,000.00",0.00,1.00,0.00,"180,000.00",inf
15,U0000003273,[Harold Simmons Foundation],"[SIMMONS, ANNETTE]","138,500.00","5,646,100.00",0.02,0.98,"-5,507,600.00",-0.98
11,U0000002706,"[[24T Contribution], Ariel Corp]","[BUCHWALD-WRIGHT, KAREN MS, WRIGHT, KAREN, WRI...","137,100.00","34,300.00",0.80,0.20,"102,800.00",3.00
30,U0000003690,"[[24T Contribution], Uline Inc, None]","[UIHLEIN, RICHARD, UIHLEIN, RICHARD E, UIHLEIN...","135,000.00","446,000.00",0.23,0.77,"-311,000.00",-0.70


#### How much did they give?

In [113]:
print("2011 inside-bill 2nd quarter total: $" + str(donors_11_2nd_qtr["total_inside_bill"].sum()))
print("2011 outside-bill 2nd quarter total: $" + str(donors_11_2nd_qtr["total_outside_bill"].sum()))

2011 inside-bill 2nd quarter total: $2840199.0
2011 outside-bill 2nd quarter total: $18940802.0


### 2009

In [114]:
inside_bill_donors_09_2nd_qtr = inside_bill_donors_committees_09_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
outside_bill_donors_09_2nd_qtr = outside_bill_donors_committees_09_2nd_qtr.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()

In [115]:
donors_09_2nd_qtr = inside_bill_donors_09_2nd_qtr.merge(outside_bill_donors_09_2nd_qtr, how="outer", on="match_id", suffixes=["_inside_bill", "_outside_bill"])
donors_09_2nd_qtr.drop(["organizations_outside_bill", "contributors_outside_bill"], axis=1, inplace=True)
donors_09_2nd_qtr.rename(columns={"organizations_inside_bill": "organizations", "contributors_inside_bill": "contributors"}, inplace=True)
donors_09_2nd_qtr["total_outside_bill"].fillna(0, inplace=True)
donors_09_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132 entries, 0 to 131
Data columns (total 5 columns):
match_id              132 non-null object
organizations         76 non-null object
contributors          76 non-null object
total_inside_bill     76 non-null float64
total_outside_bill    132 non-null float64
dtypes: float64(2), object(3)
memory usage: 6.2+ KB


In [116]:
donors_09_2nd_qtr["pct_inside_bill"] = donors_09_2nd_qtr["total_inside_bill"] / (donors_09_2nd_qtr["total_outside_bill"] + donors_09_2nd_qtr["total_inside_bill"])
donors_09_2nd_qtr["pct_outside_bill"] = donors_09_2nd_qtr["total_outside_bill"] / (donors_09_2nd_qtr["total_outside_bill"] + donors_09_2nd_qtr["total_inside_bill"])
donors_09_2nd_qtr["change"] = donors_09_2nd_qtr["total_inside_bill"] - donors_09_2nd_qtr["total_outside_bill"]
donors_09_2nd_qtr["pct_change"] = (donors_09_2nd_qtr["total_inside_bill"] - donors_09_2nd_qtr["total_outside_bill"]) / donors_09_2nd_qtr["total_outside_bill"].abs()
donors_09_2nd_qtr.sort_values("total_inside_bill", ascending=False).head()

,match_id,organizations,contributors,total_inside_bill,total_outside_bill,pct_inside_bill,pct_outside_bill,change,pct_change
15,U0000003168,[TAMKO Building Products],"[HUMPHREYS, DAVID, HUMPHREYS, DAVID C, HUMPHRE...","74,600.00","62,800.00",0.54,0.46,"11,800.00",0.19
8,U0000000310,"[Adelson Drug Clinic, Interface Group, Las Veg...","[ADELSON, MIRIAM DR, ADELSON, MIRIAM DR DR, AD...","70,400.00","73,600.00",0.49,0.51,"-3,200.00",-0.04
22,U0000003463,"[[24T Contribution], Exoxemis Inc]","[STEPHENS, JACKSON T JR]","50,400.00","174,406.00",0.22,0.78,"-124,006.00",-0.71
19,U0000003273,[Harold Simmons Foundation],"[SIMMONS, ANNETTE]","47,200.00","91,500.00",0.34,0.66,"-44,300.00",-0.48
28,U0000003658,"[Franklin Resources, None]","[JOHNSON, ANN, JOHNSON, ANN L, JOHNSON, ANN L ...","45,400.00","62,200.00",0.42,0.58,"-16,800.00",-0.27


In [117]:
print("2009 inside-bill 2nd quarter total: $" + str(donors_09_2nd_qtr["total_inside_bill"].sum()))
print("2009 outside-bill 2nd quarter total: $" + str(donors_09_2nd_qtr["total_outside_bill"].sum()))

2009 inside-bill 2nd quarter total: $888939.0
2009 outside-bill 2nd quarter total: $3713898.0


## How did each contributor's giving pattern change in each cycle compared with 2017?

### 2015

In [118]:
donors_15_17_2nd_qtr = donors_17_2nd_qtr.merge(donors_15_2nd_qtr, how="outer", on="match_id", suffixes=["_17", "_15"])
donors_15_17_2nd_qtr.drop(["change_17", "pct_change_17", "change_15", "pct_change_15"],
            axis=1, inplace=True)
donors_15_17_2nd_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_15_17_2nd_qtr = donors_15_17_2nd_qtr[["match_id", "contributors", "organizations", "total_outside_bill_15", "total_inside_bill_15",
                 "pct_outside_bill_15", "pct_inside_bill_15", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_15_17_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 11 columns):
match_id                 217 non-null object
contributors             217 non-null object
organizations            217 non-null object
total_outside_bill_15    184 non-null float64
total_inside_bill_15     147 non-null float64
pct_outside_bill_15      147 non-null float64
pct_inside_bill_15       147 non-null float64
total_outside_bill_17    217 non-null float64
total_inside_bill_17     217 non-null float64
pct_outside_bill_17      217 non-null float64
pct_inside_bill_17       217 non-null float64
dtypes: float64(8), object(3)
memory usage: 20.3+ KB


In [119]:
donors_15_17_2nd_qtr["giving_change"] = np.where((donors_15_17_2nd_qtr["pct_inside_bill_17"] > donors_15_17_2nd_qtr["pct_inside_bill_15"]) | (donors_15_17_2nd_qtr["pct_outside_bill_15"].isnull()), "Increased",
                                  np.where(donors_15_17_2nd_qtr["pct_inside_bill_17"] < donors_15_17_2nd_qtr["pct_inside_bill_15"], "Decreased",
                                           np.where(donors_15_17_2nd_qtr["pct_inside_bill_17"] == donors_15_17_2nd_qtr["pct_inside_bill_15"], "Stayed the same",
                                                   "Other")))
donors_15_17_2nd_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_15,total_inside_bill_15,pct_outside_bill_15,pct_inside_bill_15,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Ak-Chin Indian Community,"[AK-CHIN INDIAN COMMUNITY, AK-CHIN INDIAN COMM...",[Ak-Chin Indian Community],"15,200.00","110,000.00",0.12,0.88,"6,000.00","93,400.00",0.06,0.94,Increased


### 2013

In [120]:
donors_13_17_2nd_qtr = donors_17_2nd_qtr.merge(donors_13_2nd_qtr, how="outer", on="match_id", suffixes=["_17", "_13"])
donors_13_17_2nd_qtr.drop(["change_17", "pct_change_17", "change_13", "pct_change_13"],
            axis=1, inplace=True)
donors_13_17_2nd_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_13_17_2nd_qtr = donors_13_17_2nd_qtr[["match_id", "contributors", "organizations", "total_outside_bill_13", "total_inside_bill_13",
                 "pct_outside_bill_13", "pct_inside_bill_13", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_13_17_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 11 columns):
match_id                 217 non-null object
contributors             217 non-null object
organizations            217 non-null object
total_outside_bill_13    156 non-null float64
total_inside_bill_13     113 non-null float64
pct_outside_bill_13      113 non-null float64
pct_inside_bill_13       113 non-null float64
total_outside_bill_17    217 non-null float64
total_inside_bill_17     217 non-null float64
pct_outside_bill_17      217 non-null float64
pct_inside_bill_17       217 non-null float64
dtypes: float64(8), object(3)
memory usage: 20.3+ KB


In [121]:
donors_13_17_2nd_qtr["giving_change"] = np.where((donors_13_17_2nd_qtr["pct_inside_bill_17"] > donors_13_17_2nd_qtr["pct_inside_bill_13"]) | (donors_13_17_2nd_qtr["pct_outside_bill_13"].isnull()), "Increased",
                                  np.where(donors_13_17_2nd_qtr["pct_inside_bill_17"] < donors_13_17_2nd_qtr["pct_inside_bill_13"], "Decreased",
                                           np.where(donors_13_17_2nd_qtr["pct_inside_bill_17"] == donors_13_17_2nd_qtr["pct_inside_bill_13"], "Stayed the same",
                                                   "Other")))
donors_13_17_2nd_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_13,total_inside_bill_13,pct_outside_bill_13,pct_inside_bill_13,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Ak-Chin Indian Community,"[AK-CHIN INDIAN COMMUNITY, AK-CHIN INDIAN COMM...",[Ak-Chin Indian Community],"62,800.00","49,600.00",0.56,0.44,"6,000.00","93,400.00",0.06,0.94,Increased


### 2011

In [122]:
donors_11_17_2nd_qtr = donors_17_2nd_qtr.merge(donors_11_2nd_qtr, how="outer", on="match_id", suffixes=["_17", "_11"])
donors_11_17_2nd_qtr.drop(["change_17", "pct_change_17", "change_11", "pct_change_11"],
            axis=1, inplace=True)
donors_11_17_2nd_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_11_17_2nd_qtr = donors_11_17_2nd_qtr[["match_id", "contributors", "organizations", "total_outside_bill_11", "total_inside_bill_11",
                 "pct_outside_bill_11", "pct_inside_bill_11", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_11_17_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 11 columns):
match_id                 217 non-null object
contributors             217 non-null object
organizations            217 non-null object
total_outside_bill_11    170 non-null float64
total_inside_bill_11     116 non-null float64
pct_outside_bill_11      116 non-null float64
pct_inside_bill_11       116 non-null float64
total_outside_bill_17    217 non-null float64
total_inside_bill_17     217 non-null float64
pct_outside_bill_17      217 non-null float64
pct_inside_bill_17       217 non-null float64
dtypes: float64(8), object(3)
memory usage: 20.3+ KB


In [123]:
donors_11_17_2nd_qtr["giving_change"] = np.where((donors_11_17_2nd_qtr["pct_inside_bill_17"] > donors_11_17_2nd_qtr["pct_inside_bill_11"]) | (donors_11_17_2nd_qtr["pct_outside_bill_11"].isnull()), "Increased",
                                  np.where(donors_11_17_2nd_qtr["pct_inside_bill_17"] < donors_11_17_2nd_qtr["pct_inside_bill_11"], "Decreased",
                                           np.where(donors_11_17_2nd_qtr["pct_inside_bill_17"] == donors_11_17_2nd_qtr["pct_inside_bill_11"], "Stayed the same",
                                                   "Other")))
donors_11_17_2nd_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_11,total_inside_bill_11,pct_outside_bill_11,pct_inside_bill_11,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Ak-Chin Indian Community,"[AK-CHIN INDIAN COMMUNITY, AK-CHIN INDIAN COMM...",[Ak-Chin Indian Community],0.00,"1,000.00",0.00,1.00,"6,000.00","93,400.00",0.06,0.94,Decreased


### 2009

In [124]:
donors_09_17_2nd_qtr = donors_17_2nd_qtr.merge(donors_09_2nd_qtr, how="outer", on="match_id", suffixes=["_17", "_09"])
donors_09_17_2nd_qtr.drop(["change_17", "pct_change_17", "change_09", "pct_change_09"],
            axis=1, inplace=True)
donors_09_17_2nd_qtr.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"}, inplace=True)
donors_09_17_2nd_qtr = donors_09_17_2nd_qtr[["match_id", "contributors", "organizations", "total_outside_bill_09", "total_inside_bill_09",
                 "pct_outside_bill_09", "pct_inside_bill_09", "total_outside_bill_17", "total_inside_bill_17",
                 "pct_outside_bill_17", "pct_inside_bill_17"]]
donors_09_17_2nd_qtr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 217 entries, 0 to 216
Data columns (total 11 columns):
match_id                 217 non-null object
contributors             217 non-null object
organizations            217 non-null object
total_outside_bill_09    132 non-null float64
total_inside_bill_09     76 non-null float64
pct_outside_bill_09      76 non-null float64
pct_inside_bill_09       76 non-null float64
total_outside_bill_17    217 non-null float64
total_inside_bill_17     217 non-null float64
pct_outside_bill_17      217 non-null float64
pct_inside_bill_17       217 non-null float64
dtypes: float64(8), object(3)
memory usage: 20.3+ KB


In [125]:
donors_09_17_2nd_qtr["giving_change"] = np.where((donors_09_17_2nd_qtr["pct_inside_bill_17"] > donors_09_17_2nd_qtr["pct_inside_bill_09"]) | (donors_09_17_2nd_qtr["pct_outside_bill_09"].isnull()), "Increased",
                                  np.where(donors_09_17_2nd_qtr["pct_inside_bill_17"] < donors_09_17_2nd_qtr["pct_inside_bill_09"], "Decreased",
                                           np.where(donors_09_17_2nd_qtr["pct_inside_bill_17"] == donors_09_17_2nd_qtr["pct_inside_bill_09"], "Stayed the same",
                                                   "Other")))
donors_09_17_2nd_qtr.head(1)

,match_id,contributors,organizations,total_outside_bill_09,total_inside_bill_09,pct_outside_bill_09,pct_inside_bill_09,total_outside_bill_17,total_inside_bill_17,pct_outside_bill_17,pct_inside_bill_17,giving_change
0,Ak-Chin Indian Community,"[AK-CHIN INDIAN COMMUNITY, AK-CHIN INDIAN COMM...",[Ak-Chin Indian Community],nan,nan,nan,nan,"6,000.00","93,400.00",0.06,0.94,Increased


## What proportion of donors increased the share of their annual giving that fell within the two-month period in 2017 as compared with the same period in each of the previous years?

In [126]:
print(str(donors_15_17_2nd_qtr[donors_15_17_2nd_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_15_17_2nd_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2015 and 2017.")
print(str(donors_13_17_2nd_qtr[donors_13_17_2nd_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_13_17_2nd_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2013 and 2017.")
print(str(donors_11_17_2nd_qtr[donors_11_17_2nd_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_11_17_2nd_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2011 and 2017.")
print(str(donors_09_17_2nd_qtr[donors_09_17_2nd_qtr["giving_change"] == "Increased"]["match_id"].count() / donors_09_17_2nd_qtr["match_id"].count() * 100) + "% of donors increased the share of their annual giving that fell within the two-month period between 2009 and 2017.")

82.02764976958525% of donors increased the share of their annual giving that fell within the two-month period between 2015 and 2017.
84.7926267281106% of donors increased the share of their annual giving that fell within the two-month period between 2013 and 2017.
87.09677419354838% of donors increased the share of their annual giving that fell within the two-month period between 2011 and 2017.
92.16589861751152% of donors increased the share of their annual giving that fell within the two-month period between 2009 and 2017.


## And by how much did these individuals' giving increase?

In [127]:
print("These donors' giving within the two-month period increased by $" + str(donors_15_17_2nd_qtr[donors_15_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_15_17_2nd_qtr[donors_15_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_15"].sum()) + ", from $" + str(donors_15_17_2nd_qtr[donors_15_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_15"].sum()) + " to $" + str(donors_15_17_2nd_qtr[donors_15_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2015 and 2017.")
print("These donors' giving within the two-month period increased by $" + str(donors_13_17_2nd_qtr[donors_13_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_13_17_2nd_qtr[donors_13_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_13"].sum()) + ", from $" + str(donors_13_17_2nd_qtr[donors_13_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_13"].sum()) + " to $" + str(donors_13_17_2nd_qtr[donors_13_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2013 and 2017.")
print("These donors' giving within the two-month period increased by $" + str(donors_11_17_2nd_qtr[donors_11_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_11_17_2nd_qtr[donors_11_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_11"].sum()) + ", from $" + str(donors_11_17_2nd_qtr[donors_11_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_11"].sum()) + " to $" + str(donors_11_17_2nd_qtr[donors_11_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2011 and 2017.")
print("These donors' giving within the two-month period increased by $" + str(donors_09_17_2nd_qtr[donors_09_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum() - donors_09_17_2nd_qtr[donors_09_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_09"].sum()) + ", from $" + str(donors_09_17_2nd_qtr[donors_09_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_09"].sum()) + " to $" + str(donors_09_17_2nd_qtr[donors_09_17_2nd_qtr["giving_change"] == "Increased"]["total_inside_bill_17"].sum()) + " between 2009 and 2017.")

These donors' giving within the two-month period increased by $24389803.0, from $9480863.0 to $33870666.0 between 2015 and 2017.
These donors' giving within the two-month period increased by $35387586.0, from $1442575.0 to $36830161.0 between 2013 and 2017.
These donors' giving within the two-month period increased by $33393717.0, from $1354150.0 to $34747867.0 between 2011 and 2017.
These donors' giving within the two-month period increased by $35731694.0, from $649739.0 to $36381433.0 between 2009 and 2017.


## How many donors in each year gave all of their money in the two-month period of each year?

In [128]:
print("In 2017, " + str(donors_17_2nd_qtr[donors_17_2nd_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2015, " + str(donors_15_2nd_qtr[donors_15_2nd_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2013, " + str(donors_13_2nd_qtr[donors_13_2nd_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2011, " + str(donors_11_2nd_qtr[donors_11_2nd_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")
print("In 2009, " + str(donors_09_2nd_qtr[donors_09_2nd_qtr["pct_inside_bill"] == 1]["match_id"].count()) + " donors gave all their money in the two-month period.")

In 2017, 40 donors gave all their money in the two-month period.
In 2015, 5 donors gave all their money in the two-month period.
In 2013, 8 donors gave all their money in the two-month period.
In 2011, 9 donors gave all their money in the two-month period.
In 2009, 5 donors gave all their money in the two-month period.


# What does pre- and post-bill giving look like on a contribution-by-contribution level?

In [129]:
def run_contributions_query(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start, end):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_contributions_""" + suffix + """ AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN left(contribid, 11) IN ('d1007764625',
                                              'q0001024006',
                                              'U0000000002',
                                              'U0000000003',
                                              'U0000004292',
                                              'U0000004308',
                                              'U0000004652') THEN 'DEVOS'
                 WHEN left(contribid, 11) IN ('U0000003449',
                                              'U0000003450') THEN 'FERTITTA'
                 WHEN left(contribid, 11) IN ('U0000003477',
                                              'U0000003492') THEN 'KOCH'
                 WHEN left(contribid, 11) IN ('h3001352487',
                                              'm0001331786') THEN 'HILDEBRAND'
                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '""" + reference_start + """'
        AND date <= '""" + reference_end + """'
        AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN left(contribid, 11) IN ('d1007764625',
                                                              'q0001024006',
                                                              'U0000000002',
                                                              'U0000000003',
                                                              'U0000004292',
                                                              'U0000004308',
                                                              'U0000004652') THEN 'DEVOS'
                                 WHEN left(contribid, 11) IN ('U0000003449',
                                                              'U0000003450') THEN 'FERTITTA'
                                 WHEN left(contribid, 11) IN ('U0000003477',
                                                              'U0000003492') THEN 'KOCH'
                                 WHEN left(contribid, 11) IN ('h3001352487',
                                                              'm0001331786') THEN 'HILDEBRAND'
                                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE orgname
                             END = gop_donors.match_id
   WHERE date >= '""" + start + """'
     AND date <= '""" + end + """'
     AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '""" + cycle + """'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
  AND match_id NOT IN ('Air Line Pilots Assn',
                       'America First Policies',
                       'American Action Network',
                       'a0000720833',
                       'q0000962134',
                       'Republican Governors Assn',
                       'Southeast Wood Treating')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;


SELECT *
FROM """ + prefix + """_contributions_""" + suffix + """;"""
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [130]:
post_bill_contributions_17 = run_contributions_query("post_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-11-02", "2017-12-31")
pre_bill_contributions_17 = run_contributions_query("pre_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-01-01", "2017-11-01")

In [131]:
post_bill_contributions_15 = run_contributions_query("post_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-11-02", "2015-12-31")
pre_bill_contributions_15 = run_contributions_query("pre_bill", "15", "2018", "2017-11-02", "2017-12-31", "2016", "2015-01-01", "2015-11-01")

In [132]:
post_bill_contributions_13 = run_contributions_query("post_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-11-02", "2013-12-31")
pre_bill_contributions_13 = run_contributions_query("pre_bill", "13", "2018", "2017-11-02", "2017-12-31", "2014", "2013-01-01", "2013-11-01")

In [133]:
post_bill_contributions_11 = run_contributions_query("post_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-11-02", "2011-12-31")
pre_bill_contributions_11 = run_contributions_query("pre_bill", "11", "2018", "2017-11-02", "2017-12-31", "2012", "2011-01-01", "2011-11-01")

In [134]:
post_bill_contributions_09 = run_contributions_query("post_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-11-02", "2009-12-31")
pre_bill_contributions_09 = run_contributions_query("pre_bill", "09", "2018", "2017-11-02", "2017-12-31", "2010", "2009-01-01", "2009-11-01")

# What does Democratic giving look like on a contribution-by-contribution level?

In [135]:
def run_democratic_contributions_query(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start, end):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_democratic_contributions_""" + suffix + """ AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN left(contribid, 11) IN ('d1007764625',
                                              'q0001024006',
                                              'U0000000002',
                                              'U0000000003',
                                              'U0000004292',
                                              'U0000004308',
                                              'U0000004652') THEN 'DEVOS'
                 WHEN left(contribid, 11) IN ('U0000003449',
                                              'U0000003450') THEN 'FERTITTA'
                 WHEN left(contribid, 11) IN ('U0000003477',
                                              'U0000003492') THEN 'KOCH'
                 WHEN left(contribid, 11) IN ('h3001352487',
                                              'm0001331786') THEN 'HILDEBRAND'
                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '""" + reference_start + """'
        AND date <= '""" + reference_end + """'
        AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN left(contribid, 11) IN ('d1007764625',
                                                              'q0001024006',
                                                              'U0000000002',
                                                              'U0000000003',
                                                              'U0000004292',
                                                              'U0000004308',
                                                              'U0000004652') THEN 'DEVOS'
                                 WHEN left(contribid, 11) IN ('U0000003449',
                                                              'U0000003450') THEN 'FERTITTA'
                                 WHEN left(contribid, 11) IN ('U0000003477',
                                                              'U0000003492') THEN 'KOCH'
                                 WHEN left(contribid, 11) IN ('h3001352487',
                                                              'm0001331786') THEN 'HILDEBRAND'
                                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE orgname
                             END = gop_donors.match_id
   WHERE date >= '""" + start + """'
     AND date <= '""" + end + """'
     AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '""" + cycle + """'
WHERE primcode IN ('J1200',
                   'J2100',
                   'J2300',
                   'Z1200',
                   'Z4200',
                   'Z5200',
                   'Z4400')
  AND match_id NOT IN ('Air Line Pilots Assn',
                       'America First Policies',
                       'American Action Network',
                       'a0000720833',
                       'q0000962134',
                       'Republican Governors Assn',
                       'Southeast Wood Treating')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;


SELECT *
FROM """ + prefix + """_democratic_contributions_""" + suffix + """;"""
    return pd.read_sql(query, con=conn)

## Return the data for 2017.

In [136]:
post_bill_democratic_contributions_17 = run_democratic_contributions_query("post_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-11-02", "2017-12-31")
pre_bill_democratic_contributions_17 = run_democratic_contributions_query("pre_bill", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-01-01", "2017-11-01")

# What does first quarter giving look like on a contribution-by-contribution level?

In [137]:
def run_contributions_query_1st_qtr(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start_one, end_one, start_two, end_two):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_contributions_""" + suffix + """ AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN left(contribid, 11) IN ('d1007764625',
                                              'q0001024006',
                                              'U0000000002',
                                              'U0000000003',
                                              'U0000004292',
                                              'U0000004308',
                                              'U0000004652') THEN 'DEVOS'
                 WHEN left(contribid, 11) IN ('U0000003449',
                                              'U0000003450') THEN 'FERTITTA'
                 WHEN left(contribid, 11) IN ('U0000003477',
                                              'U0000003492') THEN 'KOCH'
                 WHEN left(contribid, 11) IN ('h3001352487',
                                              'm0001331786') THEN 'HILDEBRAND'
                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '""" + reference_start + """'
        AND date <= '""" + reference_end + """'
        AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN left(contribid, 11) IN ('d1007764625',
                                                              'q0001024006',
                                                              'U0000000002',
                                                              'U0000000003',
                                                              'U0000004292',
                                                              'U0000004308',
                                                              'U0000004652') THEN 'DEVOS'
                                 WHEN left(contribid, 11) IN ('U0000003449',
                                                              'U0000003450') THEN 'FERTITTA'
                                 WHEN left(contribid, 11) IN ('U0000003477',
                                                              'U0000003492') THEN 'KOCH'
                                 WHEN left(contribid, 11) IN ('h3001352487',
                                                              'm0001331786') THEN 'HILDEBRAND'
                                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE orgname
                             END = gop_donors.match_id
   WHERE date >= '""" + start_one + """'
     AND date <= '""" + end_one + """'
     OR (date >= '""" + start_two + """'
         AND date <= '""" + end_two + """')
     AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '""" + cycle + """'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
  AND match_id NOT IN ('Air Line Pilots Assn',
                       'America First Policies',
                       'American Action Network',
                       'a0000720833',
                       'q0000962134',
                       'Republican Governors Assn',
                       'Southeast Wood Treating')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;


SELECT *
FROM """ + prefix + """_contributions_"""+ suffix +""";"""
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [138]:
inside_bill_contributions_17_1st_qtr = run_contributions_query_1st_qtr("inside", "17_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2018", "2017-02-02", "2017-03-31", "2017-02-02", "2017-03-31")
outside_bill_contributions_17_1st_qtr = run_contributions_query_1st_qtr("outside", "17_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2018", "2017-01-01", "2017-02-01", "2017-04-01", "2017-12-31")

In [139]:
inside_bill_contributions_15_1st_qtr = run_contributions_query_1st_qtr("inside", "15_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2016", "2015-02-02", "2015-03-31", "2015-02-02", "2015-03-31")
outside_bill_contributions_15_1st_qtr = run_contributions_query_1st_qtr("outside", "15_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2016", "2015-01-01", "2015-02-01", "2015-04-01", "2015-12-31")

In [140]:
inside_bill_contributions_13_1st_qtr = run_contributions_query_1st_qtr("inside", "13_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2014", "2013-02-02", "2013-03-31", "2013-02-02", "2013-03-31")
outside_bill_contributions_13_1st_qtr = run_contributions_query_1st_qtr("outside", "13_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2014", "2013-01-01", "2013-02-01", "2013-04-01", "2013-12-31")

In [141]:
inside_bill_contributions_11_1st_qtr = run_contributions_query_1st_qtr("inside", "11_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2012", "2011-02-02", "2011-03-31", "2011-02-02", "2011-03-31")
outside_bill_contributions_11_1st_qtr = run_contributions_query_1st_qtr("outside", "11_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2012", "2011-01-01", "2011-02-01", "2011-04-01", "2011-12-31")

In [142]:
inside_bill_contributions_09_1st_qtr = run_contributions_query_1st_qtr("inside", "09_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2010", "2009-02-02", "2009-03-31", "2009-02-02", "2009-03-31")
outside_bill_contributions_09_1st_qtr = run_contributions_query_1st_qtr("outside", "09_1st_quarter", "2018", "2017-02-02", "2017-03-31", "2010", "2009-01-01", "2009-02-01", "2009-04-01", "2009-12-31")

# What does second quarter giving look like on a contribution-by-contribution level?

In [143]:
def run_contributions_query_2nd_qtr(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start_one, end_one, start_two, end_two):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_contributions_""" + suffix + """ AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN left(contribid, 11) IN ('d1007764625',
                                              'q0001024006',
                                              'U0000000002',
                                              'U0000000003',
                                              'U0000004292',
                                              'U0000004308',
                                              'U0000004652') THEN 'DEVOS'
                 WHEN left(contribid, 11) IN ('U0000003449',
                                              'U0000003450') THEN 'FERTITTA'
                 WHEN left(contribid, 11) IN ('U0000003477',
                                              'U0000003492') THEN 'KOCH'
                 WHEN left(contribid, 11) IN ('h3001352487',
                                              'm0001331786') THEN 'HILDEBRAND'
                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '""" + reference_start + """'
        AND date <= '""" + reference_end + """'
        AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN left(contribid, 11) IN ('d1007764625',
                                                              'q0001024006',
                                                              'U0000000002',
                                                              'U0000000003',
                                                              'U0000004292',
                                                              'U0000004308',
                                                              'U0000004652') THEN 'DEVOS'
                                 WHEN left(contribid, 11) IN ('U0000003449',
                                                              'U0000003450') THEN 'FERTITTA'
                                 WHEN left(contribid, 11) IN ('U0000003477',
                                                              'U0000003492') THEN 'KOCH'
                                 WHEN left(contribid, 11) IN ('h3001352487',
                                                              'm0001331786') THEN 'HILDEBRAND'
                                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN'
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE orgname
                             END = gop_donors.match_id
   WHERE date >= '""" + start_one + """'
     AND date <= '""" + end_one + """'
     OR (date >= '""" + start_two + """'
         AND date <= '""" + end_two + """')
     AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '""" + cycle + """'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
  AND match_id NOT IN ('Air Line Pilots Assn',
                       'America First Policies',
                       'American Action Network',
                       'a0000720833',
                       'q0000962134',
                       'Republican Governors Assn',
                       'Southeast Wood Treating')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;


SELECT *
FROM """ + prefix + """_contributions_""" + suffix + """;"""
    return pd.read_sql(query, con=conn)

## Return the data for each cycle.

In [144]:
inside_bill_contributions_committees_17_2nd_qtr = run_contributions_query_2nd_qtr("inside", "17_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2018", "2017-05-02", "2017-06-30", "2017-05-02", "2017-06-30")
outside_bill_contributions_committees_17_2nd_qtr = run_contributions_query_2nd_qtr("outside", "17_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2018", "2017-01-01", "2017-05-01", "2017-07-01", "2017-12-31")

In [145]:
inside_bill_contributions_committees_15_2nd_qtr = run_contributions_query_2nd_qtr("inside", "15_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2016", "2015-05-02", "2015-06-30", "2015-05-02", "2015-06-30")
outside_bill_contributions_committees_15_2nd_qtr = run_contributions_query_2nd_qtr("outside", "15_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2016", "2015-01-01", "2015-05-01", "2015-07-01", "2015-12-31")

In [146]:
inside_bill_contributions_committees_13_2nd_qtr = run_contributions_query_2nd_qtr("inside", "13_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2014", "2013-05-02", "2013-06-30", "2013-05-02", "2013-06-30")
outside_bill_contributions_committees_13_2nd_qtr = run_contributions_query_2nd_qtr("outside", "13_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2014", "2013-01-01", "2013-05-01", "2013-07-01", "2013-12-31")

In [147]:
inside_bill_contributions_committees_11_2nd_qtr = run_contributions_query_2nd_qtr("inside", "11_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2012", "2011-05-02", "2011-06-30", "2011-05-02", "2011-06-30")
outside_bill_contributions_committees_11_2nd_qtr = run_contributions_query_2nd_qtr("outside", "11_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2012", "2011-01-01", "2011-05-01", "2011-07-01", "2011-12-31")

In [148]:
inside_bill_contributions_committees_09_2nd_qtr = run_contributions_query_2nd_qtr("inside", "09_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2010", "2009-05-02", "2009-06-30", "2009-05-02", "2009-06-30")
outside_bill_contributions_committees_09_2nd_qtr = run_contributions_query_2nd_qtr("outside", "09_2nd_quarter", "2018", "2017-05-02", "2017-06-30", "2010", "2009-01-01", "2009-05-01", "2009-07-01", "2009-12-31")

## Individual giving patterns

### Doug Deason

In [149]:
def run_deason_contributions_query(prefix, suffix, reference_cycle, reference_start, reference_end, cycle, start, end):
    query = """CREATE MATERIALIZED VIEW IF NOT EXISTS """ + prefix + """_contributions_""" + suffix + """ AS
SELECT fectransid, 
       match_id, 
       organization, 
       contributor, date, sum(SUM), 
                          cmte_id, 
                          pacshort AS committee
FROM 
  (SELECT fectransid, 
          match_id, 
          orgname AS organization, 
          contrib AS contributor, date, sum(amount), 
                                        cmteid AS cmte_id
   FROM 
     (SELECT CASE 
                 WHEN left(contribid, 11) IN ('d1007764625', 
                                              'q0001024006', 
                                              'U0000000002', 
                                              'U0000000003', 
                                              'U0000004292', 
                                              'U0000004308', 
                                              'U0000004652') THEN 'DEVOS' 
                 WHEN left(contribid, 11) IN ('U0000003449', 
                                              'U0000003450') THEN 'FERTITTA' 
                 WHEN left(contribid, 11) IN ('U0000003477', 
                                              'U0000003492') THEN 'KOCH' 
                 WHEN left(contribid, 11) IN ('h3001352487', 
                                              'm0001331786') THEN 'HILDEBRAND' 
                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN' 
                 WHEN trim(contribid) != '' THEN left(contribid, 11) 
                 ELSE orgname 
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '""" + reference_cycle + """' 
      WHERE primcode IN ('J1100', 
                         'J2200', 
                         'J2400', 
                         'Z1100', 
                         'Z4100', 
                         'Z4500', 
                         'Z5100')
        AND date >= '""" + reference_start + """'
        AND date <= '""" + reference_end + """'
        AND crp_contributions.cycle = '""" + reference_cycle + """'
        AND TYPE NOT IN ('11J', 
                         '15C', 
                         '15E', 
                         '15I', 
                         '15J', 
                         '15T', 
                         '18J', 
                         '19J', 
                         '30J', 
                         '30F', 
                         '31J', 
                         '31F', 
                         '32J', 
                         '32F')
        AND contrib ILIKE '%deason, doug%' 
      GROUP BY match_id) AS gop_donors
   JOIN crp_contributions ON CASE 
                                 WHEN left(contribid, 11) IN ('d1007764625', 
                                                              'q0001024006', 
                                                              'U0000000002', 
                                                              'U0000000003', 
                                                              'U0000004292', 
                                                              'U0000004308', 
                                                              'U0000004652') THEN 'DEVOS' 
                                 WHEN left(contribid, 11) IN ('U0000003449', 
                                                              'U0000003450') THEN 'FERTITTA' 
                                 WHEN left(contribid, 11) IN ('U0000003477', 
                                                              'U0000003492') THEN 'KOCH' 
                                 WHEN left(contribid, 11) IN ('h3001352487', 
                                                              'm0001331786') THEN 'HILDEBRAND' 
                                 WHEN left(contribid, 11) IN ('U0000000361') THEN 'SCHWARZMAN' 
                                 WHEN trim(contribid) != '' THEN left(contribid, 11) 
                                 ELSE orgname 
                             END = gop_donors.match_id 
   WHERE date >= '""" + start + """'
     AND date <= '""" + end + """'
     AND CYCLE = '""" + cycle + """'
     AND TYPE NOT IN ('11J', 
                      '15C', 
                      '15E', 
                      '15I', 
                      '15J', 
                      '15T', 
                      '18J', 
                      '19J', 
                      '30J', 
                      '30F', 
                      '31J', 
                      '31F', 
                      '32J', 
                      '32F') 
   GROUP BY fectransid, 
            match_id, 
            organization, 
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '""" + cycle + """' 
WHERE primcode IN ('J1100', 
                   'J2200', 
                   'J2400', 
                   'Z1100', 
                   'Z4100', 
                   'Z4500', 
                   'Z5100')
  AND match_id NOT IN ('Air Line Pilots Assn', 
                       'America First Policies', 
                       'American Action Network', 
                       'a0000720833', 
                       'q0000962134', 
                       'Republican Governors Assn', 
                       'Southeast Wood Treating')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;


SELECT *
FROM """ + prefix + """_contributions_""" + suffix + """;"""
    return pd.read_sql(query, con=conn)

In [150]:
post_bill_deason_contributions_17 = run_deason_contributions_query("post_bill_deason", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-11-02", "2017-12-31")
pre_bill_deason_contributions_17 = run_deason_contributions_query("pre_bill_deason", "17", "2018", "2017-11-02", "2017-12-31", "2018", "2017-01-01", "2017-11-01")

In [151]:
pre_bill_deason_contributions_17["sum"].sum()

33800.0

In [152]:
post_bill_deason_contributions_17[post_bill_deason_contributions_17["date"] >= datetime.date(2017, 11, 10)]["sum"].sum()

41100.0

In [153]:
post_bill_deason_contributions_17[post_bill_deason_contributions_17["date"] >= datetime.date(2017, 11, 10)].sort_values("committee", ascending=True)

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
12,4021620181505009203,U0000004607,Deason Capital Services,"DEASON, DOUGLAS R",2017-12-31,"2,700.00",C00662767,Chip Roy for Congress
6,4011120181502828659,U0000004607,None,"DEASON, DOUGLAS",2017-12-31,"1,000.00",C00283523,Dallas Entrepreneur PAC
11,4021620181505002564,U0000004607,Dcs,"DEASON, DOUG",2017-11-13,"-2,700.00",C00370650,Friends of Jeb Hensarling
14,4022320181514652974,U0000004607,Deason Capital Services,"DEASON, DOUG",2017-12-29,"2,700.00",C00658401,Greg Pence for Congress
2,2022620181515963605,U0000004607,Deason Capital Services,"DEASON, DOUG",2017-12-29,"2,700.00",C00651208,John James for Senate
3,2022720181516002096,U0000004607,Deason Capital Services,"DEASON, DOUGLAS R",2017-12-31,"2,700.00",C00652727,Josh Hawley for Senate
0,1042120180036552330,U0000004607,Deason Capital Services,"DEASON, DOUGLAS R",2017-12-29,"2,700.00",C00376939,Marsha for Senate
1,2021220181504498651,U0000004607,Deason Capital Services,"DEASON, DOUGLAS R MR",2017-12-31,"2,700.00",C00548289,Matt Rosendale for Montana
4,2022720181516008230,U0000004607,Deason Capital Services,"DEASON, DOUGLAS R MR",2017-12-28,"2,700.00",C00652131,Nicholson for Senate
5,2022720181516008398,U0000004607,Deason Capital Services,"DEASON, DOUGLAS R MR",2017-12-28,"2,700.00",C00652131,Nicholson for Senate


In [154]:
post_bill_deason_contributions_17

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
0,1042120180036552330,U0000004607,Deason Capital Services,"DEASON, DOUGLAS R",2017-12-29,"2,700.00",C00376939,Marsha for Senate
1,2021220181504498651,U0000004607,Deason Capital Services,"DEASON, DOUGLAS R MR",2017-12-31,"2,700.00",C00548289,Matt Rosendale for Montana
2,2022620181515963605,U0000004607,Deason Capital Services,"DEASON, DOUG",2017-12-29,"2,700.00",C00651208,John James for Senate
3,2022720181516002096,U0000004607,Deason Capital Services,"DEASON, DOUGLAS R",2017-12-31,"2,700.00",C00652727,Josh Hawley for Senate
4,2022720181516008230,U0000004607,Deason Capital Services,"DEASON, DOUGLAS R MR",2017-12-28,"2,700.00",C00652131,Nicholson for Senate
5,2022720181516008398,U0000004607,Deason Capital Services,"DEASON, DOUGLAS R MR",2017-12-28,"2,700.00",C00652131,Nicholson for Senate
6,4011120181502828659,U0000004607,None,"DEASON, DOUGLAS",2017-12-31,"1,000.00",C00283523,Dallas Entrepreneur PAC
7,4012320181503077640,U0000004607,Deason Capital Services,"DEASON, DOUGLAS",2017-11-20,"5,400.00",C00554113,Ratcliffe for Congress
8,4012320181503077641,U0000004607,Deason Capital Services,"DEASON, DOUGLAS",2017-11-20,"-2,700.00",C00554113,Ratcliffe for Congress
9,4012320181503077642,U0000004607,Deason Capital Services,"DEASON, DOUGLAS",2017-11-20,"2,700.00",C00554113,Ratcliffe for Congress


### Kochs

In [155]:
post_bill_contributions_17[(post_bill_contributions_17["contributor"].str.contains("koch,", case=False, na=False, regex=False)) &
                           (post_bill_contributions_17["date"] == datetime.date(2017, 11, 29))]

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
258,4011920181502962176,KOCH,Koch Industries,"KOCH, CHARLES G MR",2017-11-29,"247,700.00",C00545947,Team Ryan
259,4011920181502962178,KOCH,Koch Industries,"KOCH, ELIZABETH B MRS",2017-11-29,"247,700.00",C00545947,Team Ryan


In [156]:
pre_bill_contributions_17[(pre_bill_contributions_17["contributor"].str.contains("koch,", case=False, na=False, regex=False)) &
                           (pre_bill_contributions_17["committee"] == "National Republican Senatorial Cmte")]

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
52,1090620170035657356,KOCH,Koch Industries,"KOCH, CHARLES G",2017-06-30,"33,900.00",C00027466,National Republican Senatorial Cmte
54,1090620170035657661,KOCH,Koch Industries,"KOCH, ELIZABETH MS",2017-06-30,"33,900.00",C00027466,National Republican Senatorial Cmte
59,1090620170035659579,KOCH,Koch Industries,"KOCH, CHARLES G MR",2017-06-30,"101,700.00",C00027466,National Republican Senatorial Cmte
60,1090620170035659580,KOCH,Koch Industries,"KOCH, CHARLES G MR",2017-06-30,"101,700.00",C00027466,National Republican Senatorial Cmte
61,1090620170035659581,KOCH,Koch Industries,"KOCH, ELIZABETH MS",2017-06-30,"101,700.00",C00027466,National Republican Senatorial Cmte
62,1090620170035659582,KOCH,Koch Industries,"KOCH, ELIZABETH MS",2017-06-30,"101,700.00",C00027466,National Republican Senatorial Cmte


### Occidental Petroleum

In [157]:
post_bill_contributions_17[post_bill_contributions_17["contributor"].str.contains("occidental petroleum", case=False, na=False, regex=False)]

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
477,4021220181504474716,Occidental Petroleum,Occidental Petroleum,OCCIDENTAL PETROLEUM CORPORATION,2017-12-05,"200,000.00",C00504530,Congressional Leadership Fund
718,4022020181514122928,Occidental Petroleum,Occidental Petroleum,OCCIDENTAL PETROLEUM,2017-12-06,"100,000.00",C00571703,Senate Leadership Fund


### RAI Services

In [158]:
post_bill_contributions_17[post_bill_contributions_17["contributor"].str.contains("rai services", case=False, na=False, regex=False)]

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
483,4021220181504474726,Rai Services Co,Rai Services Co,RAI SERVICES COMPANY,2017-12-20,"250,000.00",C00504530,Congressional Leadership Fund


### Hillwood Development

In [159]:
post_bill_contributions_17[post_bill_contributions_17["contributor"].str.contains("hillwood development", case=False, na=False, regex=False)]

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
484,4021220181504474728,Hillwood Development,Hillwood Development,HILLWOOD DEVELOPMENT COMPANY LLC,2017-12-21,"500,000.00",C00504530,Congressional Leadership Fund
492,4021220181504481348,Hillwood Development,Hillwood Development,HILLWOOD DEVELOPMENT COMPANY LLC,2017-12-21,"1,000,000.00",C00487363,American Crossroads


### William Austin

In [160]:
post_bill_contributions_17[(post_bill_contributions_17["contributor"].str.contains("austin, william", case=False, na=False, regex=False)) & 
                           (post_bill_contributions_17["committee"] == "Republican National Cmte")]["sum"].sum()

75000.0

### Schweitzers

In [161]:
post_bill_contributions_17[post_bill_contributions_17["contributor"].str.contains("schweitzer,", case=False, na=False, regex=False)]

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
586,4021420181504689464,U0000003654,Schweitzer Engineering Laboratories,"SCHWEITZER, BEATRIZ",2017-12-19,"150,000.00",C00543199,McMorris Rodgers American Dream Project
587,4021420181504689465,U0000003654,Schweitzer Engineering Laboratories,"SCHWEITZER, EDMUND O III",2017-12-19,"150,000.00",C00543199,McMorris Rodgers American Dream Project


### Mike Ricatto

In [162]:
post_bill_contributions_17[post_bill_contributions_17["contributor"].str.contains("ricatto,", case=False, na=False, regex=False)]

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
660,4021520181504862780,p0005387533,Real Estate,"RICATTO, MICHAEL P MR",2017-11-27,"50,000.00",C00618389,Trump Victory


### Michael Porter

In [163]:
post_bill_contributions_17[post_bill_contributions_17["contributor"].str.contains("porter,", case=False, na=False, regex=False)]

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
704,4022020181514120624,q0000828404,Retired,"PORTER, MICHAEL",2017-11-02,"200,000.00",C00638387,ProjectRedTX
705,4022020181514120625,q0000828404,Retired,"PORTER, MICHAEL",2017-12-08,"300,000.00",C00638387,ProjectRedTX


### Geoffrey Palmer

In [164]:
post_bill_contributions_17[post_bill_contributions_17["contributor"].str.contains("palmer,", case=False, na=False, regex=False)]

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
321,4012420181503199287,U0000004054,GH Palmer Assoc,"PALMER, GEOFFREY H",2017-11-03,"2,000,000.00",C00637512,America First Action
818,4022620181515628736,U0000004054,GH Palmer Assoc,"PALMER, GEOFF",2017-12-29,"-22,400.00",C00075820,National Republican Congressional Cmte
819,4022620181515628738,U0000004054,Ghpalmer Assoc,"PALMER, GEOFF",2017-12-29,"-22,400.00",C00075820,National Republican Congressional Cmte


In [165]:
post_bill_contributions_11[post_bill_contributions_11["contributor"].str.contains("palmer,", case=False, na=False, regex=False)]

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
119,4020220121150320929,U0000004054,GH Palmer Assoc,"PALMER, GEOFF MR",2011-11-22,"1,000.00",C00496497,Newt 2012
123,4020220121150380045,U0000004054,GH Palmer Assoc,"PALMER, GEOFF",2011-12-30,"-1,000.00",C00496497,Newt 2012
192,4041920121154896477,U0000004054,GH Palmer Assoc,"PALMER, GEOFF",2011-11-03,"1,000.00",C00496067,Friends of Herman Cain


### Stephen Schwarzman

In [166]:
post_bill_contributions_17[(post_bill_contributions_17["contributor"].str.contains("schwarzman, stephen", case=False, na=False, regex=False)) & 
                           (post_bill_contributions_17["committee"] == "Trump Victory")]

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
661,4021520181504862808,SCHWARZMAN,Blackstone Group,"SCHWARZMAN, STEPHEN A MR",2017-12-14,"344,400.00",C00618389,Trump Victory


### Ronald Lauder

In [167]:
post_bill_contributions_17[post_bill_contributions_17["contributor"].str.contains("lauder,", case=False, na=False, regex=False)]

,fectransid,match_id,organization,contributor,date,sum,cmte_id,committee
659,4021520181504862669,a0000115053,Estee Lauder Companies,"LAUDER, RONALD S HON",2017-11-30,"100,000.00",C00618389,Trump Victory


### Jose Pepe Fanjul

In [168]:
donors_all_years[donors_all_years["contributors"].str.join(", ").str.contains("fanjul, ", case=False, na=False, regex=False)]

,match_id,organizations,contributors,total_post_bill_09,total_pre_bill_09,pct_post_bill_09,pct_pre_bill_09,total_post_bill_11,total_pre_bill_11,pct_post_bill_11,pct_pre_bill_11,total_post_bill_13,total_pre_bill_13,pct_post_bill_13,pct_pre_bill_13,total_post_bill_15,total_pre_bill_15,pct_post_bill_15,pct_pre_bill_15,total_post_bill_17,total_pre_bill_17,pct_post_bill_17,pct_pre_bill_17
22,U0000004560,[Florida Crystals],"[FANJUL, JOSE PEPE MR SR, FANJUL, JOSE SR]","13,200.00","21,100.00",0.38,0.62,"40,000.00","27,400.00",0.59,0.41,"2,500.00","21,400.00",0.10,0.90,620.00,"25,420.00",0.02,0.98,"105,400.00","40,000.00",0.72,0.28


### John Hess

In [169]:
donors_all_years[donors_all_years["contributors"].str.join(", ").str.contains("hess,", case=False, na=False, regex=False)]

,match_id,organizations,contributors,total_post_bill_09,total_pre_bill_09,pct_post_bill_09,pct_pre_bill_09,total_post_bill_11,total_pre_bill_11,pct_post_bill_11,pct_pre_bill_11,total_post_bill_13,total_pre_bill_13,pct_post_bill_13,pct_pre_bill_13,total_post_bill_15,total_pre_bill_15,pct_post_bill_15,pct_pre_bill_15,total_post_bill_17,total_pre_bill_17,pct_post_bill_17,pct_pre_bill_17
83,a0000258995,[Hess Corp],"[HESS, JOHN B MR]",0.00,0.00,0.00,0.00,"10,000.00","1,000.00",0.91,0.09,0.00,"30,000.00",0.00,0.00,"103,300.00","95,800.00",0.52,0.48,"100,000.00","344,200.00",0.23,0.77


### Wynns

In [170]:
post_bill_contributions_17[post_bill_contributions_17["contributor"].str.contains("wynn,", case=False, na=False, regex=False)]["sum"].sum()

455123.0

In [171]:
post_bill_contributions_17[(post_bill_contributions_17["contributor"].str.contains("wynn,", case=False, na=False, regex=False)) & 
                          (post_bill_contributions_17["committee"] == "National Republican Senatorial Cmte")]["sum"].sum()

100000.0

In [172]:
post_bill_contributions_17[(post_bill_contributions_17["contributor"].str.contains("wynn,", case=False, na=False, regex=False)) & 
                          (post_bill_contributions_17["committee"] == "Republican National Cmte") & 
                          (post_bill_contributions_17["date"] == datetime.date(2017, 12, 14))]["sum"].sum()

375000.0

### Uihleins

In [173]:
donors_all_years[donors_all_years["contributors"].str.join(", ").str.contains("uihlein,", case=False, na=False, regex=False)]

,match_id,organizations,contributors,total_post_bill_09,total_pre_bill_09,pct_post_bill_09,pct_pre_bill_09,total_post_bill_11,total_pre_bill_11,pct_post_bill_11,pct_pre_bill_11,total_post_bill_13,total_pre_bill_13,pct_post_bill_13,pct_pre_bill_13,total_post_bill_15,total_pre_bill_15,pct_post_bill_15,pct_pre_bill_15,total_post_bill_17,total_pre_bill_17,pct_post_bill_17,pct_pre_bill_17
17,U0000003690,"[U-Line Corp, Uline Inc]","[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH MRS, U...","10,100.00","28,650.00",0.26,0.74,"385,200.00","188,300.00",0.67,0.33,"261,400.00","533,150.00",0.33,0.67,"-2,936,300.00","8,270,600.00",-0.55,1.55,"4,557,300.00","12,051,400.00",0.27,0.73


### Ansarys

In [174]:
donors_all_years[donors_all_years["contributors"].str.join(", ").str.contains("ansary,", case=False, na=False, regex=False)]

,match_id,organizations,contributors,total_post_bill_09,total_pre_bill_09,pct_post_bill_09,pct_pre_bill_09,total_post_bill_11,total_pre_bill_11,pct_post_bill_11,pct_pre_bill_11,total_post_bill_13,total_pre_bill_13,pct_post_bill_13,pct_pre_bill_13,total_post_bill_15,total_pre_bill_15,pct_post_bill_15,pct_pre_bill_15,total_post_bill_17,total_pre_bill_17,pct_post_bill_17,pct_pre_bill_17
65,U0000004576,[Stewart & Stevenson],"[ANSARY, HUSHANG, ANSARY, HUSHANG HON, ANSARY,...",0.00,"47,100.00",0.00,0.00,0.00,"142,400.00",0.00,0.00,"70,600.00",400.00,0.99,0.01,"129,800.00","2,924,100.00",0.04,0.96,"894,200.00","841,900.00",0.52,0.48


## Export data to Excel.

In [175]:
writer = pd.ExcelWriter("data/megadonors.xlsx")
donors_17_increased.to_excel(writer, "donors_17_increased", startcol=0, index=False)
donors_17_gave_all_last_two_months.to_excel(writer, "donors_17_last_2_months", startcol=0, index=False)
donors_15_17.to_excel(writer, "donors_15_17", startcol=0, index=False)
donors_13_17.to_excel(writer, "donors_13_17", startcol=0, index=False)
donors_11_17.to_excel(writer, "donors_11_17", startcol=0, index=False)
donors_09_17.to_excel(writer, "donors_09_17", startcol=0, index=False)
post_bill_donors_committees_17.to_excel(writer, "post_bill_donors_committees_17", startcol=0, index=False)
pre_bill_donors_committees_17.to_excel(writer, "pre_bill_donors_committees_17", startcol=0, index=False)
post_bill_donors_committees_15.to_excel(writer, "post_bill_donors_committees_15", startcol=0, index=False)
pre_bill_donors_committees_15.to_excel(writer, "pre_bill_donors_committees_15", startcol=0, index=False)
post_bill_donors_committees_13.to_excel(writer, "post_bill_donors_committees_13", startcol=0, index=False)
pre_bill_donors_committees_13.to_excel(writer, "pre_bill_donors_committees_13", startcol=0, index=False)
post_bill_donors_committees_11.to_excel(writer, "post_bill_donors_committees_11", startcol=0, index=False)
pre_bill_donors_committees_11.to_excel(writer, "pre_bill_donors_committees_11", startcol=0, index=False)
post_bill_donors_committees_09.to_excel(writer, "post_bill_donors_committees_09", startcol=0, index=False)
pre_bill_donors_committees_09.to_excel(writer, "pre_bill_donors_committees_09", startcol=0, index=False)
post_bill_contributions_17.to_excel(writer, "post_bill_contributions_17", startcol=0, index=False)
pre_bill_contributions_17.to_excel(writer, "pre_bill_contributions_17", startcol=0, index=False)
post_bill_contributions_15.to_excel(writer, "post_bill_contributions_15", startcol=0, index=False)
pre_bill_contributions_15.to_excel(writer, "pre_bill_contributions_15", startcol=0, index=False)
post_bill_contributions_13.to_excel(writer, "post_bill_contributions_13", startcol=0, index=False)
pre_bill_contributions_13.to_excel(writer, "pre_bill_contributions_13", startcol=0, index=False)
post_bill_contributions_11.to_excel(writer, "post_bill_contributions_11", startcol=0, index=False)
pre_bill_contributions_11.to_excel(writer, "pre_bill_contributions_11", startcol=0, index=False)
post_bill_contributions_09.to_excel(writer, "post_bill_contributions_09", startcol=0, index=False)
pre_bill_contributions_09.to_excel(writer, "pre_bill_contributions_09", startcol=0, index=False)
democratic_donors_17.to_excel(writer, "democratic_donors_17", startcol=0, index=False)
post_bill_democratic_donors_committees_17.to_excel(writer, "post_bill_dem_donors_cmtes_17", startcol=0, index=False)
pre_bill_democratic_donors_committees_17.to_excel(writer, "pre_bill_dem_donors_cmtes_17", startcol=0, index=False)
post_bill_democratic_contributions_17.to_excel(writer, "post_bill_dem_contributions_17", startcol=0, index=False)
pre_bill_democratic_contributions_17.to_excel(writer, "pre_bill_dem_contributions_17", startcol=0, index=False)
writer.save()